# 第8章: 幻覚を避ける

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## セットアップ

次のセットアップセルを実行して、APIキーをロードし、`get_completion`ヘルパー関数を確立します。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
from botocore.exceptions import ClientError
import json

# utilsパッケージからhintsモジュールをインポート
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None, prefill=None):
    inference_config = {
        "temperature": 0.0,
        "maxTokens": 200
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    if prefill:
        converse_api_params["messages"].append({"role": "assistant", "content": [{"text": prefill}]})
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"クライアントエラーが発生しました: {message}")

---

## レッスン

悪いニュースがあります: **Claudeは時々「幻覚」を起こし、真実でないまたは正当化されていない主張をすることがあります**。良いニュースは: 幻覚を最小限に抑えるために使用できる技術があることです。

以下では、これらの技術のいくつかを説明します。具体的には:
- Claudeに質問の答えがわからないと言う選択肢を与える
- Claudeに答える前に証拠を探すように頼む

しかし、**幻覚を避けるための多くの方法があります**。このコースで既に学んだ多くの技術も含まれています。Claudeが幻覚を起こす場合は、複数の技術を試してClaudeの精度を向上させるようにしてください。

### 例

ここに一般的な事実に関する質問がありますが、**Claudeはできるだけ役に立とうとするためにいくつかの大きなカバを幻覚として見ます**。

In [ ]:
# プロンプト
PROMPT = "Who is the heaviest hippo of all time?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

ここで試すことができる解決策は、「**Claudeに逃げ道を与える**」ことです。つまり、Claudeに答えを拒否しても良いこと、または実際に確信を持って答えを知っている場合にのみ答えるように伝えることです。

In [ ]:
# プロンプト
PROMPT = "Who is the heaviest hippo of all time? Only answer if you know the answer with certainty."

# Claudeの応答を出力します
print(get_completion(PROMPT))

以下のプロンプトでは、Claudeにユーザーの質問にほぼ関連する「気を散らす情報」を含む長い文書を提供します。**助けを求めることなく、Claudeは気を散らす情報に引っかかり**、2020年5月31日時点でのMatterportの加入者数について誤った「幻覚的な」回答をします。

**注意:** 次の章で学ぶように、**質問はテキストや文書の後に配置するのがベストプラクティスですが、ここではプロンプトを読みやすくするために上部に配置しています。プロンプトセルをダブルクリックして、完全なプロンプトテキストを取得することができます（非常に長いです！）。

In [ ]:
# プロンプト
PROMPT = """<question>2020年5月31日の正確な日付におけるMatterportの加入者数は何人でしたか？</question>
以下の文書をお読みください。その後、<answer>タグ内に簡潔な数値の回答を書いてください。

<document>
Matterport SEC filing 10-K 2023
Item 1. Business
Our Company
Matterportは、構築された世界のデジタル化とデータ化をリードしています。私たちは、構築された世界のデジタル変革が、人々が建物や周囲の物理的空間と相互作用する方法を根本的に変えると信じています。
2011年の設立以来、Matterportの先駆的な技術は、オンラインで建物、空間、場所をデジタル化、アクセス、管理するための基準を設定してきました。当社のプラットフォームの革新的なソフトウェア、空間データ駆動のデータサイエンス、3Dキャプチャ技術は、世界最大の資産クラスである建物や物理的空間を長年にわたりオフラインで未活用のままにしていた障壁を打破しました。私たちは、構築された世界のデジタル化とデータ化が、重要な運用効率と不動産価値を解き放ち続けると信じており、Matterportがこの巨大なグローバル変革をリードするプラットフォームであると考えています。
世界は急速にオフラインからオンラインへと移行しています。デジタル変革は、今日のすべてのビジネスや業界に強力で持続的な影響を与えています。国際データ公社（IDC）によると、2020年から2023年の間にデジタル変革に対して6.8兆ドル以上の直接投資が行われるとされ、2026年にはグローバルなデジタル変革支出が3.4兆ドルに達し、5年間の年平均成長率（CAGR）は16.3%になると予測されています。また、デジタルツインへの投資は、5年間のCAGRが35.2%になると予想されています。この世俗的なシフトに伴い、物理的なものからデジタルへの移行に対する需要も高まっています。それにもかかわらず、建物や空間の大多数は依然としてオフラインで未デジタル化のままです。2021年時点でSavillsによって推定されたグローバルな建物ストックは327兆ドルの総不動産価値があり、今日でも大部分がオフラインのままであり、デジタル変革によって浸透しているのは0.1%未満と推定しています。
Matterportは、構築された世界のデジタル化の必要性と、建物や空間の理解を促進するための空間データの力を認識した最初の企業の一つです。過去には、技術が物理的な地図をデータが豊富なデジタル地図や位置情報サービスに進化させました。Matterportは現在、建物をデジタル化し、キャプチャしたすべての空間の理解と潜在能力を大幅に向上させるデータが豊富な環境を作り出しています。私たちがボタンを押すだけで、ある都市から別の都市への最速ルートを瞬時に学んだり、最寄りのコーヒーショップを見つけたりできるように、Matterportの建物に関する空間データは、世界中の不動産や空間に関する豊富な洞察と学びを解き放ちます。さらに、今日の地理空間マッピングプラットフォームが業界にマッピングデータを開放し、ライドシェアリング、eコマース、フードデリバリーマーケットプレイス、さらには短期レンタルやホームシェアリングなどの新しいビジネスモデルを創出しているのと同様に、Matterportの構造化された空間データへのオープンアクセスは、ホスピタリティ、施設管理、保険、建設、不動産、小売などの新しい機会とビジネスモデルを可能にしています。
私たちは、構築された世界のデジタル化に対する総アドレス可能市場機会が2400億ドルを超え、市場がスケールで成熟するにつれて1兆ドルに達する可能性があると信じています。これは、世界中の40億以上の不動産と200億の空間のグローバルな建物ストックに関する私たちの分析、モデリング、理解に基づいています。人工知能（AI）、機械学習（ML）、深層学習（DL）技術の助けを借りて、私たちは、強力な空間データ駆動の不動産洞察と分析からの追加の収益化機会を考慮すると、構築された世界のデジタル化とデータ化のための総アドレス可能市場が1兆ドルを超えると信じています。

私たちの空間データプラットフォームとデジタルツインのキャプチャは、多様な業界やユースケースにわたって価値を提供します。大手小売業者は、数千の店舗をリモートで管理でき、不動産代理店は、数百の物件と数千の訪問者のためにバーチャルオープンハウスを提供でき、プロパティ開発者は、建設プロセス全体をより詳細かつ迅速に監視でき、保険会社は、効率的かつ正確に請求やアンダーライティングの評価を文書化し評価できます。Matterportは、世界中のほぼすべてのサイズ、形状、場所の不動産に関して、加入者にとって重要なデジタル体験、ツール、情報を提供します。
私たちは、ほぼ10年間、空間データプラットフォームを成長させ、最も詳細で正確かつデータが豊富なデジタルツインを作成するための能力を拡大してきました。さらに、私たちの3D再構築プロセスは完全に自動化されており、私たちのソリューションは、世界中のあらゆるタイプ、形状、サイズの数百万の建物や空間に対して同じ精度でスケールすることができます。私たちのサービスの普遍的な適用性は、Matterportに多様な垂直市場や地理にわたる重要なスケールとリーチを提供します。2022年12月31日現在、私たちの加入者数は、2021年12月31日現在の503,000人から約39%増加し、701,000人を超え、私たちのデジタルツインは170以上の国に達しました。私たちは、複数の業界で280億平方フィート以上の空間をデジタル化しており、市場全体に対して重要なスケールと成長を示しています。

私たちが世界中の建物をデータに変換し続ける中で、私たちは空間データプラットフォームを拡張し、業界全体の加入者のために不動産の計画、開発、管理、インテリジェンスをさらに変革し、構築された世界のための事実上のビルおよびビジネスインテリジェンスエンジンになることを目指しています。私たちは、不動産、建築、エンジニアリングおよび建設（AEC）、小売、保険、政府を含む業界全体の企業、ビジネス、機関に対する空間データとそれに伴う洞察の需要が急速に成長し続けると信じています。
私たちは、デジタル化とデータ化が、この巨大なカテゴリと資産クラス全体にわたる成長のための素晴らしいグリーンフィールドの機会を表していると信じています。設計と開発の初期段階から、マーケティング、運営、保険、建物の修理とメンテナンスに至るまで、私たちのプラットフォームのソフトウェアと技術は、加入者にコスト削減、収益増加、建物や空間の最適な管理を推進するための重要なツールと洞察を提供します。私たちは、構築された世界における未実現の利用と運用効率の数千億ドルが、私たちの空間データプラットフォームの力によって解き放たれると信じています。私たちのプラットフォームとデータソリューションは、業界や建物カテゴリ全体に普遍的な適用性を持ち、Matterportにとって大きな市場機会の全体を対象にし、世界で最大の資産クラスの価値を高めることができるという重要な利点を提供します。
加入者に価値を提供する実績を持つ私たちの提供物には、ソフトウェアサブスクリプション、データライセンス、サービス、製品ハードウェアが含まれます。2022年12月31日現在、私たちの加入者数には、フォーチュン1000企業の24%以上が含まれており、上位10社からの総収益の10%未満が生成されています。2025年までに、私たちの収益の80%以上がソフトウェアサブスクリプションとデータライセンスソリューションから来ると予想しています。私たちの革新的な3Dキャプチャ製品であるPro2およびPro3カメラは、3D建物および不動産ビジュアライゼーションエコシステムの形成に重要な役割を果たしています。Pro2およびPro3カメラは、私たちのソリューションの採用を促進し、Cortex、私たちの独自のAIソフトウェアエンジンがデジタルツインの作成のための先駆的なエンジンとなるために必要なユニークで高品質なスケールデータセットを生成しました。このデータの利点は、最初にPro2カメラによって促進され、私たちはキャプチャデバイスに依存しないプラットフォームを開発し、業界や地理にわたって加入者のために新しい建物や不動産の洞察を生成できるようにしました。
私たちは最近、急速な成長を経験しました。私たちの加入者数は、2018年12月31日から2022年12月31日までに約49倍に増加しました。私たちの収益は、2022年12月31日までの年度で約22%増加し、1億3610万ドルに達しました。2021年12月31日までの年度では約1億1120万ドルでした。私たちの粗利益は、2022年12月31日までの年度で810万ドル、または14%減少し、5180万ドルになりました。これは主に、2022会計年度におけるグローバルなサプライチェーンの制約による特定の混乱的および追加コストに起因しています。既存の加入者から生成されるサブスクリプション収益を保持し、成長させる能力は、私たちのビジネスの健全性と将来の成長の見通しを測る重要な指標です。この分野でのパフォーマンスを追跡するために、私たちは同じ顧客の同様の期間にわたるネットドル拡張率を測定しています。2022年12月31日までの3か月間のネットドル拡張率は103%であり、私たちのプラットフォームの粘着性と成長の可能性を示しています。
私たちの業界と市場機会
今日、建物や空間の大多数は未デジタル化のままです。私たちは、現在のサービス可能なアドレス市場が、主に不動産および旅行・ホスピタリティセクターから、世界中で約13億の空間を含むと推定しています。2022年12月31日現在、約920万の空間を管理しており、私たちはグローバルな建物ストックに浸透し続け、住宅および商業不動産、施設管理、小売、AEC、保険および修理、旅行およびホスピタリティなどのさまざまなエンド市場にわたって足跡を拡大しています。私たちは、総アドレス可能市場が世界中で40億以上の建物と200億の空間を超え、2400億ドル以上の市場機会を生み出すと推定しています。Matterportのユニークな空間データライブラリと不動産データサービスが成長し続けるにつれて、この機会は、建物ストックの規模と世界中の建物に対する未開発の価値創造に基づいて、1兆ドルを超える可能性があります。COVID-19パンデミックによって生じた制約は、過去10年間にわたって多様な業界や市場のために開発したスケールされた3Dキャプチャソリューションの重要性を強化し、加速させるだけでした。

私たちの空間データプラットフォーム
概要
私たちの技術プラットフォームは、さまざまなデジタルキャプチャデバイスから収集された空間データを使用して、物理的な建物や空間を寸法的に正確でフォトリアリスティックなデジタルツインに変換し、加入者に以前は利用できなかった建物情報や洞察へのアクセスを提供します。
この巨大な市場で約10年間の先駆者として、私たちは、Cortexという独自のAI駆動ソフトウェアエンジンによって強化された業界をリードする3D再構築技術を開発し、スケールしました。Cortexは、機械学習を使用して、建物全体の構造、内容、設備、家具を含むフォトリアリスティックな3D仮想表現を再現します。完成品は、物理的な空間の詳細で動的な複製であり、スマートフォンを含む任意のデバイスのウェブブラウザから探索、分析、カスタマイズできます。大規模な商業ビルを管理する力は、各加入者の手のひらにあり、私たちの高度な技術と空間データ技術スタック全体にわたる画期的な革新によって可能になっています。
私たちの空間データプラットフォームの重要な要素は以下の通りです：
•オフラインの建物をオンラインにする。従来、私たちの顧客は、建物や空間を理解し評価するために対面での現地訪問を行う必要がありました。写真や間取り図は役立つことがありますが、これらの2次元（2D）表現は情報が限られており、静的で硬直しているため、各建物や空間の全体的な理解に必要なインタラクティブな要素が欠けています。CortexのAI駆動機能により、物理的なオブジェクトの表現はもはや静的な2D画像に制限されず、物理的な訪問を排除できます。Cortexは、建物や空間をオフラインからオンラインに移動させ、顧客がどこからでもリアルタイムでオンデマンドでアクセスできるようにします。加入者が建物をスキャンした後、私たちの視覚化アルゴリズムは、スキャンを通じてキャプチャされた平面の2D画像から空間の位置と深さを正確に推測し、それらを高忠実度で正確なデジタルツインモデルに変換します。これにより、各デジタルツインがプロフェッショナルグレードの画像品質であることを保証する完全自動化された画像処理パイプラインが作成されます。
•空間データによって駆動される。私たちはデータ駆動型の企業です。空間の各インクリメンタルキャプチャは、私たちの空間データライブラリの豊かさと深さを増します。空間データは、人間が作り出した環境における建物や空間を構成するユニークで特異な詳細を表します。Cortexは、私たちがこれまでに蓄積した数十億のデータポイントの幅を利用して、デジタルツインの3D精度を向上させます。私たちは、加入者がデジタルツイン内のオブジェクトの高さ、位置、その他の特性を特定できるように支援します。私たちの高度なアルゴリズムは、加入者にデータに基づく洞察を生成することによって重要な商業的価値を提供します。たとえば、プロパティ開発者は特定の窓からの自然熱や日光の量を評価でき、小売業者は各店舗のレイアウトが同じレベルのコードやブランド要件を満たしていることを確認でき、工場は機械のレイアウトが仕様や位置ガイドラインを満たしていることを確認できます。2022年12月31日現在、約920万の空間を管理しており、私たちの空間データライブラリは構築された世界に関する情報のクリアリングハウスです。
•AIとMLによって駆動される。人工知能と機械学習技術は、空間データを効果的に利用して、動的でリアルなインタラクティブな体験を作成します。AIとMLは、日常的なスキャンに高価なカメラを必要としなくします。加入者は、スマートフォンのボタンをタップするだけで空間をスキャンできるようになりました。その結果、Matterportはデバイスに依存しないプラットフォームとなり、私たちの構築された世界のデジタル化とインデックス化の使命に向けて迅速にスケールし、推進しています。
私たちの加入者に対する価値提案は、設計と構築からメンテナンスと運営、プロモーション、販売、リース、保険、修理、復元、セキュリティ、資金調達に至るまで、デジタルビルライフサイクル全体にサービスを提供するように設計されています。その結果、私たちは、加入者が建物や空間をオフラインからオンラインに移行する機会を発見するのを助けることで、短期的および長期的な投資収益率を推進する機会を見出すことができるため、収益を成長させるためのユニークな立場にあると信じています。
ユビキタスキャプチャ
Matterportは3D空間キャプチャの標準となりました。私たちの技術プラットフォームは、世界中の加入者が迅速かつ簡単に、正確に建物や空間のインタラクティブで寸法的に正確なデジタルツインをデジタル化、カスタマイズ、管理できるようにします。
Matterportプラットフォームは、すべての加入者のキャプチャニーズに応じて、さまざまなLiDAR、球面、3D、360カメラ、およびスマートフォンと連携するように設計されています。これにより、任意のサイズ、スケール、複雑さの空間を、いつでもどこでもキャプチャする柔軟性が提供されます。
•Matterport Pro3は、プロジェクトの完了を加速するために、以前のバージョンよりも速くプロパティをスキャンする最新の3Dカメラです。Pro3は、屋内および屋外の空間の最高の精度のスキャンを提供し、速度、忠実度、多様性、精度のために設計されています。100メートル離れた場所で3Dデータをキャプチャし、スイープごとに20秒未満で、Pro3の超高速で高精度なLiDARセンサーは数時間動作し、あらゆる条件下で数百万の測定を行います。
•Matterport Pro2は、世界中で数百万の空間を高い忠実度、精度、速度、シンプルさでキャプチャするために使用されている独自の3Dカメラです。50万平方フィートを超える建物をキャプチャでき、多くの住宅、商業、工業、大規模プロパティの選択肢となっています。
•360カメラ。Matterportは、市場で利用可能な360カメラの選択肢をサポートしています。これらの手頃な価格のポケットサイズのデバイスは、高忠実度で精密なキャプチャを提供し、小さな家、コンドミニアム、短期レンタル、アパートなどのキャプチャに適しています。これらのデバイスの球面レンズ画像キャプチャ技術は、Cortexに堅牢で詳細な画像データを提供し、パノラマを業界をリードするデジタルツインに変換します。
•LEICA BLK360。Leicaとの提携を通じて、私たちの3D再構築技術とAI駆動のソフトウェアエンジンであるCortexは、この強力なLiDARカメラをMatterportデジタルツインを作成するための超精密なキャプチャデバイスに変換します。正確な精度が求められる場合、AEC専門家にとっての選択肢です。
•スマートフォンキャプチャ。私たちのキャプチャアプリは、iOSおよびAndroidの両方で商業的に利用可能です。Matterportのスマートフォンキャプチャソリューションは、3Dキャプチャを民主化し、最近のiPhoneデバイスを使用して建物や空間をデジタル化することを簡単かつアクセス可能にしました。2020年5月にMatterport for iPhoneの初回導入以来、2021年4月にはAndroid Captureアプリの公式リリースを発表し、Androidユーザーが没入型の3Dで建物や空間を迅速かつ簡単にキャプチャできるようにしました。2022年2月には、Matterport Axisを立ち上げ、スマートフォンを保持するモーター付きマウントを使用して、Matterport Captureアプリで任意の物理空間の3Dデジタルツインをより迅速、正確、一貫してキャプチャできるようにしました。
Cortexと3D再構築（Matterportデジタルツイン）
2022年12月31日現在、約920万の空間を管理し、約280億平方フィートのキャプチャされた空間を表す空間データライブラリを持つ私たちは、先進的なMLおよびDL技術を使用して、キャプチャした空間データをアルゴリズム的に正確な3Dデジタル再現に変換します。このインテリジェントで自動化された3D再構築は、Cortexによって可能になり、私たちの空間データライブラリを使用して、建物や空間がどのようにフロアや部屋に分かれているか、ドアや開口部の位置、どのような部屋が存在するかを理解します。そのため、これらの形状は寸法的に正確に動的でフォトリアリスティックなデジタルツインに編纂され、整列されます。Cortexの他のコンポーネントには、建物や空間内の内容を特定し分類するためのAI駆動のコンピュータビジョン技術や、家具や設備からドア、窓、照明器具、火災抑制スプリンクラー、避難経路に至るまでのすべてを特定しセグメント化するためのオブジェクト認識技術が含まれます。私たちの高度にスケーラブルな人工知能プラットフォームは、加入者がボタンをクリックするだけで強力で強化された建物データと洞察にアクセスできるようにします。

Matterportデジタルツインの背後にある科学：Cortex AIのハイライト
MatterportはCortexで動作します
Cortexは、任意の建物や空間のデジタルツインを作成するための精密な深層学習ニューラルネットワークを含むAI駆動のソフトウェアエンジンです。Pro2およびPro3カメラでキャプチャされた独自の空間データを使用して開発され、Cortexは高い精度と正確性を提供し、日常的なデバイスを使用した3Dキャプチャを可能にします。
一般的なニューラルネットワークは、現実世界の3D再構築に苦労します。Matterport最適化ネットワークは、より正確で堅牢な結果を提供します。生のトレーニングデータ以上のものであり、Matterportのデータセットは、新しいニューラルネットワークアーキテクチャを開発し、数百万の状況におけるユーザー行動や実世界のデータに対して評価することを可能にします。
•深層学習：各空間の詳細なニューラルネットワークデータアーキテクチャを接続し最適化することは、堅牢で非常に正確な3Dデジタルツインを作成するための鍵です。Cortexは、数百万の建物や空間から集約されたMatterportの豊富な空間データに対して各3Dモデルを評価し最適化します。Cortexの評価能力と3D再構築のデータ駆動最適化は、さまざまな建物構成、空間、環境にわたって一貫した高精度の結果を生み出します。
•動的3D再構築：2Dビジュアルや静的画像からスケールで正確な3D空間データを作成するには、複数の視点からのフォトリアリスティックで詳細なデータと数百万の空間が必要です。これにより、Cortexのニューラルネットワークと学習能力が改善され、任意の空間の3D再構築が最適化されます。Cortexの機能と、私たちの3Dキャプチャ技術におけるリアルタイム空間整列アルゴリズムを組み合わせることで、進行中の作業の直感的な「プレビュー」を作成し、加入者がコンテンツをインタラクティブかつリアルタイムで操作できるようにします。
•コンピュータビジョン：Cortexは、デジタルツインの価値を高めるための強力な機能のスイートを提供します。これには、部屋や部屋内のオブジェクトの自動測定、自動2Dから3Dへの高解像度フォトギャラリーの作成、プライバシー保護のための自動顔ぼかし、カスタムビデオ、ウォークスルー、自動部屋ラベリング、オブジェクト認識が含まれます。
•高度な画像処理：Matterportの計算写真アルゴリズムは、各デジタルツインがプロフェッショナルグレードの画像品質であることを保証するための完全自動化された画像処理パイプラインを作成します。私たちの特許技術により、3Dキャプチャはボタンを押すだけで簡単になります。Matterportのソフトウェアと技術は、ホワイトバランスやカメラ特有の色補正、高ダイナミックレンジトーンマッピング、ノイズ除去、霞除去、シャープ化、彩度、その他の調整を含む残りのステップを管理します。
空間データとAI駆動の洞察
すべてのMatterportデジタルツインには、建物、部屋、または物理空間に関する広範な情報が含まれています。このデータは、私たちのAI駆動のCortexエンジンを使用しています。Matterportデジタルツイン自体に加えて、私たちの空間データは、精密な建物のジオメトリと構造の詳細、建物の内容、設備、状態、空間内の多くの視点からの高解像度の画像とフォトリアリスティックな詳細で構成されています。Cortexは、私たちが「深い空間インデックス」と呼ぶ技術を使用しています。深い空間インデックスは、人工知能、コンピュータビジョン、深層学習を使用して、各空間、その構造、および内容に関する重要な詳細を正確かつ忠実に特定し伝達します。私たちは、Matterportの加入者が任意の建物のための相互運用可能なデジタル記録システムを活用できるようにする堅牢な空間データ標準を作成しました。
デジタルツインの構築を通じて加入者に対して高度にインタラクティブなデジタル体験を提供するだけでなく、私たちはすべての加入者に対して2つの質問を自問します：（1）彼らの建物や物理空間にとって重要なことは何か、（2）この空間に対してどのような学びや洞察を提供できるか？私たちのAI駆動のCortexエンジンは、私たちの空間データライブラリを使用して、集約されたプロパティトレンドや運用および評価の洞察を提供することで、これらの質問に答えるのを助けます。さらに、Matterportプラットフォームエコシステムが拡大し続けるにつれて、私たちの加入者、パートナー、他のサードパーティ開発者は、私たちの豊富な空間データレイヤーから収穫できる洞察の幅と深さをさらに広げるために独自のツールを持ち込むことができます。
拡張可能なプラットフォームエコシステム
Matterportは、2022年12月31日現在、約920万の空間を管理し、約280億平方フィートのキャプチャされた空間を持つ、世界で最も大きく、最も正確な空間データライブラリを提供しています。私たちの空間データプラットフォームの多様性と広範なエンタープライズソフトウェア開発キットおよびアプリケーションプログラミングインターフェース（API）は、地理や垂直市場によってMatterportの価値提案を拡張する堅牢なグローバルエコシステムのチャネルとパートナーを開発することを可能にしました。私たちは、ターゲット市場全体で統合されたMatterportソリューションを促進するために、パートナーやその加入者との広範なワークフロー統合を展開し続けるつもりです。また、私たちは、簡単に展開でき、簡単にアクセスできるMatterportソフトウェアアドオンで空間データプラットフォームの力を拡張するためのサードパーティソフトウェアマーケットプレイスを開発しています。このマーケットプレイスは、開発者が新しいアプリケーションや空間データマイニングツールを構築し、Matterportの3D体験を強化し、コア提供物を補完する新しい生産性および不動産管理ツールを作成できるようにします。サードパーティ開発者によって作成されたこれらの付加価値機能は、スケーラブルな新しい収益源を可能にし、Matterportはオンラインマーケットプレイスを通じて加入者に展開される各アドオンからのサブスクリプションおよびサービス収益を共有します。私たちのプラットフォームエコシステムのネットワーク効果は、私たちのビジネスの成長に寄与し、加入者の粘着性とユーザーエンゲージメントを高めることで将来の成長を支えると信じています。
Matterportのアドオンや拡張機能の例には以下が含まれます：
•アドオン：Encircle（迅速な請求処理のための使いやすいフィールドドキュメンテーションツール）；WP Matterport Shortcode（Matterportを迅速かつ簡単に埋め込むことを可能にする無料のWordpressプラグイン）、WP3D Models（WordPress + Matterport統合プラグイン）；Rela（リスティングのためのオールインワンマーケティングソリューション）；CAPTUR3D（Matterportデジタルツインに価値を拡張するオールインワンコンテンツ管理システム）；Private Model Emded（企業が追加のユーザーライセンスを必要とせずに、企業ネットワーク上の大規模な従業員グループとデジタルツインをプライベートに共有できる機能）；Views（グループや大規模組織が異なるチームで異なるタスクを管理するための別々の権限ベースのワークフローを作成できる新しい作業グループコラボレーションフレームワーク）；およびGuided Tours and Tags（訪問者の興味に合わせた商業または住宅空間の指向されたバーチャルツアーを作成するためのツール）。私たちは、企業顧客がIoTデータを視覚的に没入型で寸法的に正確なMatterportデジタルツインにシームレスに接続できるようにするために、Amazon Web Services（AWS）IoT TwinMakerとのプライベートベータ統合を発表しました。
•サービス：Matterport ADA準拠デジタルツイン（アメリカ障害者法に準拠したデジタルツインを提供するソリューション）およびエンタープライズクラウドソフトウェアプラットフォーム（任意のサイズや形状の建物や空間のデジタルツインを作成、公開、管理するための再構築されたクラウドソフトウェアプラットフォーム）。
私たちの競争力の強み
私たちは、市場リーダーシップを成長させるための競争力の強みがいくつかあると信じています。私たちの競争力の強みには以下が含まれます：
•Matterportプラットフォームの幅と深さ。私たちのコアの強みは、カスタマイズなしでキャプチャから処理、業界全体にわたる広範なリーチを持つオールインワンの空間データプラットフォームです。さまざまなエンタープライズシステムとシームレスに統合できる能力を持つ私たちのプラットフォームは、不動産、AEC、旅行・ホスピタリティ、修理・保険、工業・施設などの多様なエンド市場にわたって、プロパティライフサイクル全体に価値を提供します。2022年12月31日現在、私たちのグローバルなリーチは170以上の国の加入者に拡大し、フォーチュン1000企業の24%以上を含んでいます。
•市場リーダーシップと先駆者の利点。Matterportは、ほぼ10年前に構築された世界のデジタル化とデータ化のカテゴリを定義し、このカテゴリのグローバルリーダーとなりました。2022年12月31日現在、私たちのプラットフォームには701,000人以上の加入者があり、約920万の空間を管理しています。私たちのリーダーシップは、デジタルツインの作成における先駆者であったことに主に起因しています。先駆者の利点の結果、私たちは空間データの深く豊かなライブラリを蓄積し続けており、それが私たちのリーダーシップの地位を強化しています。
•重要なネットワーク効果。新しいキャプチャやデータがプラットフォームに追加されるたびに、データセットの豊かさと管理されている空間からの洞察の深さが増します。さらに、データを洞察に変える能力と、加入者による新しいデータキャプチャからの追加データの組み合わせにより、Matterportは加入者向けの機能を開発できます。私たちは、構築された世界のための空間データライブラリを構築する先駆者であり、データから洞察を収集し導出するリーダーシップは、ますます強化され、その洞察の関連性が新しい加入者を引き付けます。
•貴重なプロパティ洞察のための原材料としての巨大な空間データライブラリ。私たちの空間データライブラリの規模は、加入者のための洞察を導出する上で重要な利点です。私たちの空間データライブラリは、Cortexの重要な真実の基盤として機能し、Matterportが低コストのデジタルおよびスマートフォンカメラを使用して強力な3Dデジタルツインを作成できるようにします。2022年12月31日現在、私たちのデータは約920万の空間から管理され、約280億平方フィートがキャプチャされています。その結果、私たちはプロパティの洞察と分析を新たなレベルに引き上げ、さまざまな業界の加入者に利益をもたらしています。たとえば、施設管理者は建物のレイアウトを作成するのに必要な時間を大幅に短縮し、現地調査や実測モデリングのコストを大幅に削減します。AECの加入者は、各実測空間の分析を使用して文書化を効率化し、簡単にコラボレーションできます。
•グローバルなリーチとスケール。私たちは、AI駆動の空間データプラットフォームを世界中に拡大し続けることに注力しています。私たちは、北米、ヨーロッパ、アジアに重要な存在感を持ち、各地域にリーダーシップチームと市場投入インフラを持っています。ロンドン、シンガポール、アメリカ合衆国のいくつかの都市にオフィスを構え、国際的な拡大を加速しています。2022年12月31日現在、170以上の国に701,000人以上の加入者がいます。私たちの空間データプラットフォームの地理に依存しない性質は、国際的に成長し続ける上で重要な利点であると信じています。
•10年間の研究開発と革新を支える広範な特許ポートフォリオ。2022年12月31日現在、54件の特許が発行され、37件の特許出願が保留中です。私たちの成功は、革新に対するほぼ10年間の焦点に基づいています。革新はMatterportの中心にあり、私たちは市場のリーディングポジションをさらに強化するために研究開発への投資を優先し続けます。
•優れたキャプチャ技術。Matterportのキャプチャ技術プラットフォームは、建物や空間のMatterportデジタルツインを作成するために必要なさまざまなキャプチャデバイスをサポートするソフトウェアフレームワークです。
これには、LiDARカメラ、360カメラ、スマートフォン、Matterport Axis、Matterport Pro2およびPro3カメラのサポートが含まれます。Pro2カメラは、私たちの空間データの利点の基盤となり、Matterport対応のサードパーティキャプチャデバイスの配列でその利点を拡大しました。2022年8月、私たちはPro3カメラの発売を開始し、業界をリードするデジタルツインクラウドプラットフォームに大規模なアップデートを行いました。Matterport Pro3カメラは、高速な起動時間、交換可能なバッテリー、軽量設計を備えた高度な3Dキャプチャデバイスです。Pro3カメラは、屋内外の両方で機能し、速度、忠実度、多様性、精度のために設計されています。Pro2カメラとともに、私たちはPro3カメラの将来の販売が私たちのソリューションの採用を引き続き促進すると期待しています。Matterportは3Dキャプチャ体験を民主化し、高忠実度で高精度な3Dデジタルツインを、あらゆる建物タイプや不動産ライフサイクルのニーズに対して容易に利用できるようにしています。他の3Dキャプチャソリューションプロバイダーは存在しますが、真の寸法的に正確な3D結果を生成できるものは非常に少なく、フォトリアリスティックな3Dで最終製品を自動的に作成できるものはさらに少ないです。この広範なキャプチャ技術の提供は、スマートフォンや360カメラでキャプチャされた写真から正確なデジタルツインを自動的に生成するためにAI駆動のCortexエンジンをトレーニングするために利用可能な豊富な空間データライブラリがなければ不可能です。
</document>"""


これをどう修正しますか？さて、長い文書での幻覚を減らすための素晴らしい方法は、**Claudeにまず証拠を集めさせることです。**

この場合、私たちは**Claudeに関連する引用をまず抽出させ、その引用に基づいて回答を作成させる**ように指示します。ここでClaudeにそうするように指示することで、引用が質問に答えていないことを正しく認識させることができます。

In [ ]:
# プロンプト
PROMPT = """<question>2020年5月31日の正確な日付におけるMatterportの加入者数は何人でしたか？</question>
以下の文書をお読みください。その後、<scratchpad>タグ内に文書から最も関連性の高い引用を引き出し、それがユーザーの質問に対する答えとなるか、十分な詳細が欠けているかを考慮してください。次に、<answer>タグ内に簡潔な数値回答を書いてください。

<document>
Matterport SEC filing 10-K 2023
項目1. ビジネス
私たちの会社
Matterportは、構築された世界のデジタル化とデータ化をリードしています。私たちは、構築された世界のデジタル変革が、人々が建物や周囲の物理的空間と相互作用する方法を根本的に変えると信じています。
2011年の設立以来、Matterportの先駆的な技術は、建物、空間、場所をオンラインでデジタル化、アクセス、管理するための基準を設定してきました。私たちのプラットフォームの革新的なソフトウェア、空間データ駆動のデータサイエンス、3Dキャプチャ技術は、世界最大の資産クラスである建物や物理的空間を長年にわたりオフラインで未活用のままにしていた障壁を打破しました。私たちは、構築された世界のデジタル化とデータ化が、重要な運用効率と不動産価値を解放し続けると信じており、Matterportがこの巨大なグローバル変革をリードするプラットフォームであると考えています。
世界は急速にオフラインからオンラインへと移行しています。デジタル変革は、今日のすべてのビジネスや業界に強力で持続的な影響を与えています。国際データ公社（IDC）によると、2020年から2023年の間にデジタル変革に対して6.8兆ドル以上の直接投資が行われ、2026年にはグローバルなデジタル変革支出が3.4兆ドルに達し、5年間の年平均成長率（CAGR）は16.3%に達すると予測されています。また、デジタルツインへの投資は、5年間のCAGRが35.2%に達すると予想されています。この世俗的なシフトに伴い、物理的なものからデジタルへの移行に対する需要も高まっています。それにもかかわらず、建物や空間の大多数は依然としてオフラインで未デジタル化のままです。2021年時点でサヴィルズによって推定されたグローバルな建物ストックは327兆ドルの総不動産価値があり、今日でも大部分がオフラインのままです。私たちは、デジタル変革によって浸透しているのは0.1%未満であると推定しています。
Matterportは、構築された世界のデジタル化の必要性と、建物や空間の理解を促進するための空間データの力を認識した最初の企業の一つでした。過去には、技術が物理的な地図をデータが豊富なデジタル地図や位置情報サービスに進化させました。Matterportは現在、建物をデジタル化し、キャプチャしたすべての空間の理解と潜在能力を大幅に向上させるデータが豊富な環境を作り出しています。私たちがボタンを押すだけで、ある都市から別の都市への最速ルートを瞬時に学んだり、最寄りのコーヒーショップを見つけたりできるように、Matterportの建物に関する空間データは、世界中の不動産や空間に関する豊富な洞察と学びを解放します。さらに、今日の地理空間マッピングプラットフォームが業界にマッピングデータを開放し、ライドシェアリング、eコマース、フードデリバリーマーケットプレイス、さらには短期レンタルやホームシェアリングなどの新しいビジネスモデルを創出しているのと同様に、Matterportの構造化された空間データへのオープンアクセスは、ホスピタリティ、施設管理、保険、建設、不動産、小売などの新しい機会とビジネスモデルを可能にしています。
私たちは、構築された世界のデジタル化に対する総アドレス可能市場機会が2400億ドルを超え、市場がスケールで成熟するにつれて1兆ドルに達する可能性があると信じています。これは、世界中の40億以上の不動産と200億の空間のグローバルな建物ストックに関する私たちの分析、モデリング、理解に基づいています。人工知能（AI）、機械学習（ML）、深層学習（DL）技術の助けを借りて、私たちは、強力な空間データ駆動の不動産洞察と分析からの追加の収益化機会を考慮すると、構築された世界のデジタル化とデータ化のための総アドレス可能市場が1兆ドルを超えると信じています。

私たちの空間データプラットフォームとデジタルツインのキャプチャは、多様な業界やユースケースに価値を提供します。大手小売業者は、数千の店舗をリモートで管理でき、不動産会社は、数百の物件と数千の訪問者のためにバーチャルオープンハウスを提供でき、プロパティ開発者は、建設プロセス全体をより詳細かつ迅速に監視でき、保険会社は、効率的かつ正確に請求や引受評価を文書化し評価できます。Matterportは、世界中のほぼすべてのサイズ、形状、場所の不動産に関して、加入者にとって重要なデジタル体験、ツール、情報を提供します。
私たちは、ほぼ10年間、空間データプラットフォームを成長させ、最も詳細で正確かつデータが豊富なデジタルツインを作成するための能力を拡大してきました。さらに、私たちの3D再構築プロセスは完全に自動化されており、私たちのソリューションは、世界中のあらゆるタイプ、形状、サイズの数百万の建物や空間に対して同じ精度でスケールすることができます。私たちのサービスの普遍的な適用性は、Matterportに多様な垂直市場や地理にわたる大規模なスケールとリーチを提供します。2022年12月31日現在、私たちの加入者数は、2021年12月31日現在の503,000人から約39%増加し、701,000人を超え、私たちのデジタルツインは170カ国以上に広がっています。私たちは、複数の業界で280億平方フィート以上の空間をデジタル化しており、市場全体に対して重要なスケールと成長を示しています。

私たちが世界中の建物をデータに変換し続ける中で、私たちは空間データプラットフォームを拡張し、加入者のために物件の計画、開発、管理、インテリジェンスをさらに変革し、構築された世界のための事実上の建物およびビジネスインテリジェンスエンジンになることを目指しています。私たちは、不動産、建築、エンジニアリングおよび建設（AEC）、小売、保険、政府を含む業界全体で、企業、ビジネス、機関に対する空間データとその結果として得られる洞察の需要が急速に増加し続けると信じています。
私たちは、デジタル化とデータ化が、この巨大なカテゴリと資産クラス全体で成長のための素晴らしいグリーンフィールドの機会を表すと信じています。設計と開発の初期段階から、マーケティング、運営、保険、建物の修理とメンテナンスに至るまで、私たちのプラットフォームのソフトウェアと技術は、加入者にコスト削減、収益増加、建物や空間の最適な管理を推進するための重要なツールと洞察を提供します。私たちは、構築された世界における未実現の利用と運用効率の数千億ドルが、私たちの空間データプラットフォームの力によって解放されると信じています。私たちのプラットフォームとデータソリューションは、業界や建物カテゴリ全体に普遍的な適用性を持ち、Matterportにとって大きな利点を提供します。私たちは、この大規模な市場機会全体に対処し、世界で最大の資産クラスであると信じるものの価値を高めることができます。
加入者に価値を提供する実績を持つ私たちの提供物には、ソフトウェアサブスクリプション、データライセンス、サービス、製品ハードウェアが含まれます。2022年12月31日現在、私たちの加入者数には、フォーチュン1000企業の24%以上が含まれており、上位10社からの総収益の10%未満が生成されています。2025年までに、私たちの収益の80%以上がソフトウェアサブスクリプションとデータライセンスソリューションから来ると予想しています。私たちの革新的な3Dキャプチャ製品であるPro2およびPro3カメラは、3D建物および不動産ビジュアライゼーションエコシステムの形成に重要な役割を果たしています。Pro2およびPro3カメラは、私たちのソリューションの採用を促進し、Cortex（私たちの独自のAIソフトウェアエンジン）がデジタルツイン作成の先駆的なエンジンとなるためのユニークで高品質なスケールデータセットを生成しました。このデータの利点は、最初にPro2カメラによって促進され、私たちはキャプチャデバイスに依存しないプラットフォームを開発し、業界や地理にわたって加入者のために新しい建物や不動産の洞察を生成できるようにしました。
私たちは最近、急速な成長を経験しました。私たちの加入者数は、2018年12月31日から2022年12月31日までに約49倍に増加しました。私たちの収益は、2022年12月31日までの年度で約22%増加し、1億3610万ドルに達しました。2021年12月31日までの年度では約1億1120万ドルでした。私たちの粗利益は、2022年12月31日までの年度で810万ドル、すなわち14%減少し、5180万ドルとなりました。これは、2022会計年度におけるグローバルなサプライチェーンの制約による特定の混乱的かつ追加的なコストに起因しています。既存の加入者から生成されるサブスクリプション収益を保持し、成長させる能力は、私たちのビジネスの健全性と将来の成長見通しの重要な指標です。この分野でのパフォーマンスを追跡するために、私たちは同じ顧客セットからのネットドル拡張率を測定しています。2022年12月31日までの3ヶ月間のネットドル拡張率は103%であり、私たちのプラットフォームの粘着性と成長の可能性を示しています。
私たちの業界と市場機会
今日、建物や空間の大多数は未デジタル化のままです。私たちは、現在のサービス可能なアドレス市場が、主に不動産および旅行・ホスピタリティセクターから、世界中で約13億の空間を含むと推定しています。2022年12月31日現在、約920万の空間を管理しており、私たちはグローバルな建物ストックに浸透し、住宅および商業不動産、施設管理、小売、AEC、保険および修理、旅行およびホスピタリティを含むさまざまなエンド市場にわたって足跡を拡大しています。私たちは、総アドレス可能市場が世界中で40億以上の建物と200億の空間を超え、2400億ドル以上の市場機会を生み出すと推定しています。Matterportのユニークな空間データライブラリと不動産データサービスが成長し続けるにつれて、この機会は、建物ストックの規模と世界中の建物に対する未開発の価値創造に基づいて、1兆ドルを超える可能性があります。COVID-19パンデミックによって生じた制約は、過去10年間にわたって多様な業界や市場のために開発したスケールされた3Dキャプチャソリューションの重要性を強化し加速させました。

私たちの空間データプラットフォーム
概要
私たちの技術プラットフォームは、さまざまなデジタルキャプチャデバイスから収集された空間データを使用して、物理的な建物や空間を寸法的に正確でフォトリアリスティックなデジタルツインに変換し、加入者に以前は利用できなかった建物情報や洞察へのアクセスを提供します。
この巨大な市場でほぼ10年間の先駆者として、私たちは、Cortex（私たちの独自のAI駆動ソフトウェアエンジン）によって駆動される業界をリードする3D再構築技術を開発し、スケールしました。Cortexは、機械学習を使用して、建物構造全体のフォトリアリスティックな3D仮想表現を再現します。完成品は、物理的な空間の詳細で動的な複製であり、スマートフォンを含む任意のデバイスのウェブブラウザから探索、分析、カスタマイズできます。大規模な商業ビルを管理する力は、各加入者の手のひらにあり、私たちの先進的な技術と空間データ技術スタック全体にわたる画期的な革新によって可能になっています。
私たちの空間データプラットフォームの重要な要素は以下の通りです：
•オフラインの建物をオンラインにする。従来、私たちの顧客は、建物や空間を理解し評価するために対面での現地訪問を行う必要がありました。写真や間取り図は役立つことがありますが、これらの2次元（2D）表現は情報が限られており、静的で硬直しているため、各建物や空間の全体的な理解に重要なインタラクティブな要素が欠けています。CortexのAI駆動機能により、物理オブジェクトの表現はもはや静的な2D画像に制限されず、物理的な訪問を排除できます。Cortexは、建物や空間をオフラインからオンラインに移行し、顧客がどこからでもリアルタイムでオンデマンドでアクセスできるようにします。加入者が建物をスキャンした後、私たちの視覚化アルゴリズムは、スキャンを通じてキャプチャされた平面の2D画像から空間の位置と深さを正確に推測し、それらを高忠実度で正確なデジタルツインモデルに変換します。これにより、各デジタルツインがプロフェッショナルグレードの画像品質であることを保証する完全自動化された画像処理パイプラインが作成されます。
•空間データによって駆動される。私たちはデータ駆動型の会社です。空間の各インクリメンタルキャプチャは、私たちの空間データライブラリの豊かさと深さを増します。空間データは、人間が作り出した環境における建物や空間を構成するユニークで特異な詳細を表します。Cortexは、私たちがこれまでに蓄積した数十億のデータポイントの幅を使用して、デジタルツインの3D精度を向上させます。私たちは、加入者がデジタルツイン内のオブジェクトの高さ、位置、その他の特性を特定できるように支援します。私たちの高度なアルゴリズムは、加入者に商業的価値を提供し、彼らが自分の不動産について自信を持って評価や意思決定を行えるようにするデータベースの洞察を生成します。たとえば、プロパティ開発者は特定の窓からの自然熱や日光の量を評価し、小売業者は各店舗のレイアウトが同じレベルのコードやブランド要件を満たしていることを確認し、工場は機械のレイアウトが仕様や位置ガイドラインを満たしていることを確認できます。2022年12月31日現在、約920万の空間を管理しており、私たちの空間データライブラリは構築された世界に関する情報のクリアリングハウスです。
•AIとMLによって駆動される。人工知能と機械学習技術は、空間データを効果的に利用して、動的でリアルなインタラクティブな体験を作成し、情報を提供し、複数の視点からの表示を許可します。AIとMLは、日常的なスキャンに高価なカメラを必要としなくします。加入者は、スマートフォンのボタンをタップするだけで空間をスキャンできるようになりました。その結果、Matterportはデバイスに依存しないプラットフォームとなり、私たちの構築された世界のデジタル化とインデックス化の使命に向けてより迅速にスケールし、推進しています。
私たちの加入者に対する価値提案は、設計と構築からメンテナンスと運営、プロモーション、販売、リース、保険、修理、復元、セキュリティ、資金調達に至るまで、デジタル建物ライフサイクル全体にサービスを提供するように設計されています。その結果、私たちは、加入者が自分の建物や空間をオフラインからオンラインに移行する機会を発見するのを助けることで、短期的および長期的な投資収益率を推進する機会を見出すことができるため、収益を成長させるために独自の立場にあると信じています。
普遍的なキャプチャ
Matterportは3D空間キャプチャの標準となりました。私たちの技術プラットフォームは、世界中の加入者が迅速かつ簡単に、正確に建物や空間のインタラクティブで寸法的に正確なデジタルツインをデジタル化、カスタマイズ、管理できるようにします。
Matterportプラットフォームは、さまざまなLiDAR、球面、3D、360カメラ、スマートフォンと連携して、すべての加入者のキャプチャニーズに対応するように設計されています。これにより、あらゆるサイズ、スケール、複雑さの空間を、いつでもどこでもキャプチャする柔軟性が提供されます。
•Matterport Pro3は、プロジェクトの完了を加速するために、以前のバージョンよりも速くプロパティをスキャンする最新の3Dカメラです。Pro3は、屋内および屋外の空間の最高の精度のスキャンを提供し、速度、忠実度、多様性、精度のために設計されています。100メートル先までの3Dデータを20秒未満でキャプチャし、Pro3の超高速で高精度なLiDARセンサーは数時間動作し、あらゆる条件下で数百万の測定を行います。
•Matterport Pro2は、世界中で数百万の空間を高い忠実度、精度、速度、シンプルさでキャプチャするために使用されている独自の3Dカメラです。50万平方フィートを超える建物をキャプチャでき、多くの住宅、商業、工業、大規模プロパティの選択カメラとなっています。
•360カメラ。Matterportは、市場で利用可能な360カメラの選択肢をサポートしています。これらの手頃な価格のポケットサイズのデバイスは、高忠実度で精密なキャプチャを提供し、小さな家、コンドミニアム、短期レンタル、アパートなどのキャプチャに適しています。これらのデバイスの球面レンズ画像キャプチャ技術は、Cortexに堅牢で詳細な画像データを提供し、パノラマを業界をリードするデジタルツインに変換します。
•LEICA BLK360。ライカとの提携を通じて、私たちの3D再構築技術とAI駆動のソフトウェアエンジンであるCortexは、この強力なLiDARカメラをMatterportデジタルツインを作成するための超精密なキャプチャデバイスに変換します。正確な精度が求められるAEC専門家にとって、選択肢のソリューションです。
•スマートフォンキャプチャ。私たちのキャプチャアプリは、iOSおよびAndroidの両方で商業的に利用可能です。Matterportのスマートフォンキャプチャソリューションは、3Dキャプチャを民主化し、最近のiPhoneデバイスを使用して誰でも建物や空間をデジタル化できるようにしました。2020年5月にMatterport for iPhoneの初回導入以来、2021年4月にはAndroid Captureアプリの公式リリースを発表し、Androidユーザーが没入型の3Dで建物や空間を迅速かつ簡単にキャプチャできるようにしました。2022年2月には、Matterport Axisを立ち上げ、スマートフォンを保持するモーターマウントを使用して、Matterport Captureアプリで物理空間の3Dデジタルツインをより速く、正確に、一貫してキャプチャできるようにしました。
Cortexと3D再構築（Matterportデジタルツイン）
2022年12月31日現在、約920万の空間を管理し、約280億平方フィートの空間をキャプチャしている私たちは、先進的なMLおよびDL技術を使用して、キャプチャした空間データを正確な3Dデジタル再現にアルゴリズム的に変換します。このインテリジェントで自動化された3D再構築は、Cortex（私たちのAI駆動のソフトウェアエンジン）によって可能になり、深層学習ニューラルネットワークを含み、私たちの空間データライブラリを使用して、建物や空間がどのようにフロアや部屋に分かれているか、ドアや開口部の位置、どのような種類の部屋が存在するかを理解します。その形状は、寸法的な正確さで動的でフォトリアリスティックなデジタルツインに編纂され、整列されます。Cortexの他のコンポーネントには、建物や空間内の内容を特定し分類するためのAI駆動のコンピュータビジョン技術や、家具や設備からドア、窓、照明器具、火災抑制スプリンクラー、非常口に至るまでのすべてを特定しセグメント化するためのオブジェクト認識技術が含まれます。私たちの高度にスケーラブルな人工知能プラットフォームは、加入者がボタンをクリックするだけで強力で強化された建物データと洞察にアクセスできるようにします。

Matterportデジタルツインの背後にある科学：Cortex AIのハイライト
MatterportはCortexで動作します
Cortexは、任意の建物や空間のデジタルツインを作成するための精密な深層学習ニューラルネットワークを含むAI駆動のソフトウェアエンジンです。私たちのPro2およびPro3カメラでキャプチャされた独自の空間データを使用して開発され、Cortexは高い精度と正確性を提供し、日常的なデバイスを使用した3Dキャプチャを可能にします。
一般的なニューラルネットワークは、現実世界の3D再構築に苦労します。Matterport最適化ネットワークは、より正確で堅牢な結果を提供します。生のトレーニングデータ以上のものであり、Matterportのデータセットは、新しいニューラルネットワークアーキテクチャを開発し、数百万の状況におけるユーザー行動や実世界のデータに対して評価することを可能にします。
•深層学習：各空間の詳細なニューラルネットワークデータアーキテクチャを接続し最適化することは、堅牢で非常に正確な3Dデジタルツインを作成するための鍵です。Cortexは、数百万の建物や空間から集約されたMatterportの豊富な空間データに対して各3Dモデルを評価し最適化します。Cortexの評価能力と3D再構築のデータ駆動の最適化は、さまざまな建物構成、空間、環境にわたって一貫した高精度の結果を生み出します。
•動的3D再構築：2Dビジュアルや静的画像からスケールで正確な3D空間データを作成するには、複数の視点からのフォトリアリスティックで詳細なデータと数百万の空間が必要です。これにより、Cortexのニューラルネットワークと学習能力が改善され、任意の空間の3D再構築が最適化されます。Cortexの機能と、私たちの3Dキャプチャ技術におけるリアルタイム空間整列アルゴリズムを組み合わせることで、進行中の作業の直感的な「プレビュー」を作成し、加入者がコンテンツをインタラクティブかつリアルタイムで操作できるようにします。
•コンピュータビジョン：Cortexは、デジタルツインの価値を高めるための強力な機能のスイートを提供します。これには、部屋や部屋内のオブジェクトの自動測定、自動2Dから3Dへの高解像度フォトギャラリー作成、プライバシー保護のための自動顔ぼかし、カスタムビデオ、ウォークスルー、自動部屋ラベリング、オブジェクト認識が含まれます。
•高度な画像処理：Matterportの計算写真アルゴリズムは、各デジタルツインがプロフェッショナルグレードの画像品質であることを保証するための完全自動化された画像処理パイプラインを作成します。私たちの特許技術により、3Dキャプチャはボタンを押すだけで簡単になります。Matterportのソフトウェアと技術は、ホワイトバランスやカメラ特有の色補正、高ダイナミックレンジトーンマッピング、ノイズ除去、霞除去、シャープ化、彩度、その他の調整を含む残りのステップを管理します。
空間データとAI駆動の洞察
すべてのMatterportデジタルツインには、建物、部屋、または物理空間に関する広範な情報が含まれています。このデータは、私たちのAI駆動のCortexエンジンを使用しています。Matterportデジタルツイン自体に加えて、私たちの空間データは、精密な建物のジオメトリと構造的詳細、建物の内容、設備、状態、空間内の多くの視点からの高解像度画像とフォトリアリスティックな詳細で構成されています。Cortexは、私たちが「深い空間インデックス」と呼ぶ技術を採用しています。深い空間インデックスは、人工知能、コンピュータビジョン、深層学習を使用して、各空間、その構造、および内容に関する重要な詳細を正確かつ忠実に特定し伝達します。私たちは、Matterportの加入者が任意の建物のための相互運用可能なデジタル記録システムを活用できるようにする堅牢な空間データ標準を作成しました。
デジタルツインの構築を通じて加入者に対して高度にインタラクティブなデジタル体験を提供するだけでなく、私たちはすべての加入者に対して2つの質問を自問します：（1）彼らの建物や物理空間にとって重要なことは何か、（2）この空間に対してどのような学びや洞察を提供できるか？私たちのAI駆動のCortexエンジンは、私たちの空間データライブラリを使用して、集約されたプロパティトレンドや運用および評価の洞察を提供することで、これらの質問に答えるのを助けます。さらに、Matterportプラットフォームエコシステムが拡大し続ける中で、私たちの加入者、パートナー、他のサードパーティ開発者は、自分たちのツールを持ち込んで、私たちの豊富な空間データ層から収穫できる洞察の幅と深さをさらに広げることができます。
拡張可能なプラットフォームエコシステム
Matterportは、2022年12月31日現在、約920万の空間を管理し、約280億平方フィートをキャプチャした、世界で最も大きく、最も正確な空間データライブラリを提供しています。私たちの空間データプラットフォームの多様性と広範なエンタープライズソフトウェア開発キットおよびアプリケーションプログラミングインターフェース（API）は、Matterportの価値提案を地理的および垂直市場で拡張する堅牢なグローバルエコシステムを開発することを可能にしました。私たちは、ターゲット市場全体で統合されたMatterportソリューションを促進するために、パートナーやその加入者との広範なワークフロー統合を展開し続けるつもりです。また、私たちは、簡単に展開でき、アクセスしやすいMatterportソフトウェアアドオンで空間データプラットフォームの力を拡張するためのサードパーティソフトウェアマーケットプレイスを開発しています。このマーケットプレイスは、開発者が新しいアプリケーションや空間データマイニングツールを構築し、Matterportの3D体験を強化し、コア提供物を補完する新しい生産性およびプロパティ管理ツールを作成できるようにします。サードパーティ開発者によって作成されたこれらの付加価値機能は、スケーラブルな新しい収益源を可能にし、Matterportはオンラインマーケットプレイスを通じて加入者に展開される各アドオンからのサブスクリプションおよびサービス収益を共有します。私たちのプラットフォームエコシステムのネットワーク効果は、私たちのビジネスの成長に寄与し、加入者の粘着性とユーザーエンゲージメントを高めることで、将来の成長を支えると信じています。
Matterportのアドオンや拡張の例には以下が含まれます：
•アドオン：Encircle（迅速な請求処理のための使いやすいフィールドドキュメンテーションツール）；WP Matterport Shortcode（Matterportを迅速かつ簡単に埋め込むことを可能にする無料のWordpressプラグイン）、WP3D Models（WordPress + Matterport統合プラグイン）；Rela（リスティングのためのオールインワンマーケティングソリューション）；CAPTUR3D（Matterportデジタルツインに価値を拡張するオールインワンコンテンツ管理システム）；Private Model Embed（企業が追加のユーザーライセンスを必要とせずに、企業ネットワーク上の大規模な従業員グループとデジタルツインをプライベートに共有できる機能）；Views（グループや大規模な組織が異なるチームで異なるタスクを管理するための別々の権限ベースのワークフローを作成できる新しい作業グループコラボレーションフレームワーク）；およびガイド付きツアーとタグ（訪問者の興味に合わせた商業または住宅空間の指向されたバーチャルツアーを作成するツール）。私たちは、企業顧客がIoTデータを視覚的に没入型で寸法的に正確なMatterportデジタルツインにシームレスに接続できるようにするために、Amazon Web Services（AWS）IoT TwinMakerとのプライベートベータ統合を発表しました。
•サービス：Matterport ADA準拠デジタルツイン（アメリカ障害者法に準拠したデジタルツインを提供するソリューション）およびエンタープライズクラウドソフトウェアプラットフォーム（任意のサイズや形状の建物や空間のデジタルツインを作成、公開、管理するための再構築されたクラウドソフトウェアプラットフォーム）。
私たちの競争力の強み
私たちは、市場リーダーシップを成長させるためのいくつかの競争力の強みを持っていると信じています。私たちの競争力の強みには以下が含まれます：
•Matterportプラットフォームの幅と深さ。私たちのコアの強みは、カスタマイズなしでキャプチャから処理、業界全体にわたる広範なリーチを持つオールインワンの空間データプラットフォームです。さまざまなエンタープライズシステムとシームレスに統合できる能力を持つ私たちのプラットフォームは、不動産、AEC、旅行・ホスピタリティ、修理・保険、工業・施設などの多様なエンド市場にわたって、プロパティライフサイクル全体に価値を提供します。2022年12月31日現在、私たちのグローバルなリーチは170カ国以上の加入者に拡大し、フォーチュン1000企業の24%以上を含んでいます。
•市場リーダーシップと先行者利益。Matterportは、ほぼ10年前に構築された世界のデジタル化とデータ化のカテゴリを定義し、このカテゴリのグローバルリーダーとなりました。2022年12月31日現在、私たちは701,000人以上の加入者をプラットフォーム上に持ち、約920万の空間を管理しています。私たちのリーダーシップは、デジタルツイン作成における先行者利益によって主に推進されています。先行者利益の結果、私たちは深く豊かな空間データライブラリを蓄積し、それが私たちのリーダーシップポジションを強化し続けています。
•重要なネットワーク効果。新しいキャプチャやデータがプラットフォームに追加されるたびに、データセットの豊かさと管理されている空間からの洞察の深さが増します。さらに、データを洞察に変える能力と、加入者による新しいデータキャプチャからのインクリメンタルデータの組み合わせにより、Matterportは加入者向けの機能を開発できます。私たちは、構築された世界のための空間データライブラリを構築する先行者であり、データから洞察を収集し導き出すリーダーシップは、ますます強化され、その洞察の関連性が新しい加入者を引き寄せます。
•貴重なプロパティ洞察のための原材料としての巨大な空間データライブラリ。私たちの空間データライブラリの規模は、加入者のための洞察を導き出す上で重要な利点です。私たちの空間データライブラリは、Cortexの重要な基盤となり、Matterportが低コストのデジタルカメラやスマートフォンカメラを使用して強力な3Dデジタルツインを作成できるようにします。2022年12月31日現在、私たちのデータは約920万の空間から来ており、約280億平方フィートをキャプチャしています。その結果、私たちは不動産の洞察と分析を新たなレベルに引き上げ、さまざまな業界の加入者に利益をもたらしています。たとえば、施設管理者は建物のレイアウトを作成するのに必要な時間を大幅に短縮し、現地調査や実測モデルのコストを大幅に削減します。AECの加入者は、各実測空間の分析を使用して文書化を効率化し、簡単にコラボレーションできます。
•グローバルなリーチとスケール。私たちは、AI駆動の空間データプラットフォームを世界中に拡大し続けることに注力しています。私たちは、北米、ヨーロッパ、アジアに重要な存在感を持ち、各地域にリーダーシップチームと市場投入インフラを持っています。ロンドン、シンガポール、アメリカ各地にオフィスがあり、国際的な拡大を加速しています。2022年12月31日現在、私たちは170カ国以上で701,000人以上の加入者を持っています。私たちの空間データプラットフォームの地理的に無関係な性質は、国際的に成長し続ける上で重要な利点であると信じています。
•10年間の研究開発と革新を支える広範な特許ポートフォリオ。2022年12月31日現在、私たちは54件の特許が発行され、37件の特許出願が保留中です。私たちの成功は、ほぼ10年間の革新への注力に基づいています。革新はMatterportの中心にあり、私たちは市場のリーディングポジションをさらに進めるために研究開発への投資を優先し続けます。
•優れたキャプチャ技術。Matterportのキャプチャ技術プラットフォームは、建物や空間のMatterportデジタルツインを作成するために必要なさまざまなキャプチャデバイスをサポートするソフトウェアフレームワークです。
これには、LiDARカメラ、360カメラ、スマートフォン、Matterport Axis、Matterport Pro2およびPro3カメラのサポートが含まれます。Pro2カメラは、私たちの空間データの利点の基盤となり、Matterport対応のサードパーティキャプチャデバイスの配列でその利点を拡大しました。2022年8月、私たちはPro3カメラの発売を開始し、業界をリードするデジタルツインクラウドプラットフォームに大規模な更新を行いました。Matterport Pro3カメラは、高速な起動時間、交換可能なバッテリー、軽量設計を備えた高度な3Dキャプチャデバイスです。Pro3カメラは、屋内外の両方で機能し、速度、忠実度、多様性、精度のために設計されています。Pro2カメラとともに、私たちはPro3カメラの将来の販売が私たちのソリューションの採用を引き続き促進すると期待しています。Matterportは3Dキャプチャ体験を民主化し、高忠実度で高精度な3Dデジタルツインを、あらゆる建物タイプや加入者のニーズに対して容易に利用できるようにしています。他の3Dキャプチャソリューションプロバイダーは存在しますが、真の寸法的に正確な3D結果を生成できるものは非常に少なく、フォトリアリスティックな3Dで最終製品を自動的に作成できるものはさらに少ないです。この広範なキャプチャ技術の提供は、スマートフォンや360カメラでキャプチャされた写真から正確なデジタルツインを自動的に生成するために、AI駆動のCortexエンジンをトレーニングするために利用可能な豊富な空間データライブラリがなければ不可能です。
</document>"""


#### ボーナスレッスン

時々、Claudeの幻覚はClaudeの応答の`temperature`を下げることで解決できます。`temperature`は、0から1の間で回答の創造性を測定するもので、1はより予測不可能で標準化されていないことを意味し、0は最も一貫性があります。

`temperature`が0の状態でClaudeに何かを尋ねると、繰り返しの試行においてほぼ決定論的な回答セットが得られます（ただし、完全な決定論は保証されません）。`temperature`が1（またはその中間の段階）でClaudeに何かを尋ねると、より変動のある回答が得られます。`temperature`やその他のパラメータについての詳細は[こちら](https://docs.anthropic.com/claude/reference/messages_post)で確認できます。

上記の内容を変更せずにレッスンのプロンプトを試してみたい場合は、レッスンノートブックの一番下までスクロールして[**例のプレイグラウンド**](#example-playground)にアクセスしてください。

---

## 演習
- [演習 8.1 - ビヨンセの幻覚](#exercise-81---beyoncé-hallucination)
- [演習 8.2 - プロスペクタスの幻覚](#exercise-82---prospectus-hallucination)

### 演習 8.1 - ビヨンセの幻覚
`PROMPT`を修正して、クロードの幻覚の問題を解決し、クロードに逃げ道を与えます。（『ルネサンス』はビヨンセの7枚目のスタジオアルバムであり、8枚目ではありません。）

まず、クロードが何を幻覚するかを見るために、セルを実行することをお勧めします。その後、修正を試みてください。

In [ ]:
# Prompt
PROMPT = "スター・パフォーマーのビヨンセが彼女の8枚目のスタジオアルバムをリリースしたのは何年ですか？"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    contains = bool(
        re.search("残念ながら", text) or
        re.search("私はしない", text) or
        re.search("私はしない", text)
    )
    does_not_contain = not bool(re.search("2022", text))
    return contains and does_not_contain

# Print Claude's response and the corresponding grade
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_8_1_hint)

### 演習 8.1 - プロスペクタスの幻覚
`PROMPT`を修正して、Claudeの幻覚の問題を解決し、引用を求めるようにします。正しい答えは、加入者が49倍に増えたということです。

In [ ]:
# Get Claude's response
response = get_completion(PROMPT)

# 関数: 演習の正確性を評価する
def grade_exercise(text):
    return bool(re.search("49-fold", text))

# Claudeの応答と対応する評価を出力
print(response)
print("\n------------------------------------------ 評価 ------------------------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_8_2_hint)

### おめでとうございます！

ここまでのすべての演習を解決した場合、次の章に進む準備が整いました。楽しいプロンプト作成を！

---

## 例のプレイグラウンド

これは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのように影響するかを見るためのエリアです。

In [ ]:
# プロンプト
PROMPT = "Who is the heaviest hippo of all time?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "Who is the heaviest hippo of all time? Only answer if you know the answer with certainty."

# Claudeの応答を出力します
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = """<question>2020年5月31日の正確な日付におけるMatterportの加入者数は何人でしたか？</question>
以下の文書をお読みください。その後、<answer>タグ内に簡潔な数値の回答を書いてください。

<document>
Matterport SEC filing 10-K 2023
Item 1. Business
Our Company
Matterportは、構築された世界のデジタル化とデータ化をリードしています。私たちは、構築された世界のデジタル変革が、人々が建物や周囲の物理的空間と相互作用する方法を根本的に変えると信じています。
2011年の設立以来、Matterportの先駆的な技術は、建物、空間、場所をオンラインでデジタル化、アクセス、管理するための基準を設定してきました。当社のプラットフォームの革新的なソフトウェア、空間データ駆動のデータサイエンス、3Dキャプチャ技術は、世界最大の資産クラスである建物や物理的空間を長年にわたりオフラインで未活用のままにしていた障壁を打破しました。私たちは、構築された世界のデジタル化とデータ化が、重要な運用効率と不動産価値を解き放ち続けると信じており、Matterportがこの巨大なグローバル変革をリードするプラットフォームであると考えています。
世界は急速にオフラインからオンラインへと移行しています。デジタル変革は、今日のすべてのビジネスや業界に強力で持続的な影響を与えています。国際データ公社（IDC）によると、2020年から2023年の間にデジタル変革に対して6.8兆ドル以上の直接投資が行われるとされ、2026年にはグローバルなデジタル変革支出が3.4兆ドルに達し、5年間の年平均成長率（CAGR）は16.3%になると予測されています。また、デジタルツインへの投資は、5年間のCAGRが35.2%になると予想されています。この世俗的なシフトに伴い、物理的なものからデジタルへの移行に対する需要も高まっています。それにもかかわらず、建物や空間の大多数は依然としてオフラインで未デジタル化のままです。2021年時点でSavillsによって推定されたグローバルな建物ストックは327兆ドルの総不動産価値があり、今日でも大部分がオフラインのままであり、デジタル変革によって浸透しているのは0.1%未満と推定しています。
Matterportは、構築された世界のデジタル化の必要性と、建物や空間の理解を促進するための空間データの力をいち早く認識した企業の一つです。過去には、技術が物理的な地図をデータが豊富なデジタル地図や位置情報サービスに進化させました。Matterportは現在、建物をデジタル化し、キャプチャしたすべての空間の理解と潜在能力を大幅に向上させるデータが豊富な環境を作り出しています。私たちがボタンを押すだけで、ある都市から別の都市への最速ルートを瞬時に学んだり、最寄りのコーヒーショップを見つけたりできるように、Matterportの建物に関する空間データは、世界中の不動産や空間に関する豊富な洞察と学びを解き放ちます。さらに、今日の地理空間マッピングプラットフォームが業界にマッピングデータを開放し、ライドシェアリング、eコマース、フードデリバリーマーケットプレイス、さらには短期レンタルやホームシェアリングなどの新しいビジネスモデルを創出しているように、Matterportの構造化された空間データへのオープンアクセスは、ホスピタリティ、施設管理、保険、建設、不動産、小売などの新しい機会とビジネスモデルを可能にしています。
私たちは、構築された世界のデジタル化に対する総アドレス可能市場機会が2400億ドルを超え、市場がスケールで成熟するにつれて1兆ドルに達する可能性があると信じています。これは、世界中の40億以上の不動産と200億の空間のグローバルな建物ストックに関する私たちの分析、モデリング、理解に基づいています。人工知能（AI）、機械学習（ML）、深層学習（DL）技術の助けを借りて、私たちは、強力な空間データ駆動の不動産洞察と分析からの追加の収益化機会を考慮すると、構築された世界のデジタル化とデータ化のための総アドレス可能市場が1兆ドルを超えると信じています。

私たちの空間データプラットフォームとデジタルツインのキャプチャは、多様な業界やユースケースにわたって価値を提供します。大手小売業者は、数千の店舗をリモートで管理でき、不動産代理店は、数百の物件と数千の訪問者のためにバーチャルオープンハウスを提供でき、プロパティ開発者は、建設プロセス全体をより詳細かつ迅速に監視でき、保険会社は、効率的かつ正確に請求やアンダーライティングの評価を文書化し評価できます。Matterportは、世界中のほぼすべてのサイズ、形状、場所の不動産に関して、加入者にとって重要なデジタル体験、ツール、情報を提供します。
私たちは、ほぼ10年間、空間データプラットフォームを成長させ、最も詳細で正確かつデータが豊富なデジタルツインを作成するための能力を拡大してきました。さらに、私たちの3D再構築プロセスは完全に自動化されており、私たちのソリューションは、世界中のあらゆるタイプ、形状、サイズの数百万の建物や空間に対して同じ精度でスケールすることができます。私たちのサービスの普遍的な適用性は、Matterportに多様な垂直市場や地理にわたる大規模なスケールとリーチを提供します。2022年12月31日現在、私たちの加入者数は、2021年12月31日現在の503,000人から約39%増加し、701,000人を超え、私たちのデジタルツインは170カ国以上に広がっています。私たちは、複数の業界で280億平方フィート以上の空間をデジタル化しており、市場全体に対して重要なスケールと成長を示しています。

私たちが世界中の建物をデータに変換し続ける中で、私たちは空間データプラットフォームを拡張し、業界全体の加入者のために不動産の計画、開発、管理、インテリジェンスをさらに変革し、構築された世界のための事実上のビルディングおよびビジネスインテリジェンスエンジンになることを目指しています。私たちは、不動産、建築、エンジニアリングおよび建設（AEC）、小売、保険、政府を含む業界全体の企業、ビジネス、機関に対する空間データとそれに伴う洞察の需要が急速に成長し続けると信じています。
私たちは、デジタル化とデータ化が、この巨大なカテゴリと資産クラス全体にわたる成長のための素晴らしいグリーンフィールドの機会を表していると信じています。設計と開発の初期段階から、マーケティング、運営、保険、建物の修理とメンテナンスに至るまで、私たちのプラットフォームのソフトウェアと技術は、加入者にコスト削減、収益増加、建物や空間の最適な管理を推進するための重要なツールと洞察を提供します。私たちは、構築された世界における未実現の利用と運用効率の数千億ドルが、私たちの空間データプラットフォームの力によって解き放たれると信じています。私たちのプラットフォームとデータソリューションは、業界や建物カテゴリ全体に普遍的な適用性を持ち、Matterportにとって大きな利点を提供します。私たちは、この大規模な市場機会全体に対処し、世界で最大の資産クラスであると信じるものの価値を高めることができます。
加入者に価値を提供する実績を持つ私たちの提供物には、ソフトウェアサブスクリプション、データライセンス、サービス、製品ハードウェアが含まれます。2022年12月31日現在、私たちの加入者数には、フォーチュン1000企業の24%以上が含まれており、上位10社からの総収益の10%未満が生成されています。2025年までに、私たちの収益の80%以上がソフトウェアサブスクリプションとデータライセンスソリューションから来ると予想しています。私たちの革新的な3Dキャプチャ製品であるPro2およびPro3カメラは、3D建物および不動産ビジュアライゼーションエコシステムの形成に重要な役割を果たしてきました。Pro2およびPro3カメラは、私たちのソリューションの採用を促進し、Cortex（私たちの独自のAIソフトウェアエンジン）がデジタルツイン作成の先駆的なエンジンとなるために必要なユニークで高品質なスケールデータセットを生成しました。このデータの利点は、最初にPro2カメラによって促進され、私たちはキャプチャデバイスに依存しないプラットフォームを開発し、業界や地理にわたって加入者のために新しい建物や不動産の洞察を生成できるようにしました。
私たちは最近、急速な成長を経験しました。私たちの加入者数は、2018年12月31日から2022年12月31日までに約49倍に増加しました。私たちの収益は、2022年12月31日までの年度で約22%増加し、1億3610万ドルに達しました。2021年12月31日までの年度では約1億1120万ドルでした。私たちの粗利益は、2022年12月31日までの年度で810万ドル、14%減少し、5180万ドルになりました。これは主に、2022会計年度におけるグローバルなサプライチェーンの制約による特定の混乱的および追加コストに起因しています。既存の加入者から生成されるサブスクリプション収益を保持し、成長させる能力は、私たちのビジネスの健全性と将来の成長の見通しを測る重要な指標です。この分野でのパフォーマンスを追跡するために、私たちは同じ顧客セットからのネットドル拡張率を測定しています。2022年12月31日までの3か月間のネットドル拡張率は103%であり、私たちのプラットフォームの粘着性と成長の可能性を示しています。
私たちの業界と市場機会
今日、建物や空間の大多数は未デジタル化のままです。私たちは、現在のサービス可能なアドレス市場が、主に不動産および旅行・ホスピタリティセクターから、世界中で約13億の空間を含むと推定しています。2022年12月31日現在、約920万の空間を管理しており、私たちはグローバルな建物ストックに浸透し続け、住宅および商業不動産、施設管理、小売、AEC、保険および修理、旅行およびホスピタリティなどのさまざまなエンド市場にわたって足跡を拡大しています。私たちは、総アドレス可能市場が世界中で40億以上の建物と200億の空間を超え、2400億ドル以上の市場機会を生み出すと推定しています。Matterportのユニークな空間データライブラリと不動産データサービスが成長し続けるにつれて、この機会は、建物ストックの規模と世界中の建物に対する未開発の価値創造に基づいて、1兆ドルを超える可能性があります。COVID-19パンデミックによって生じた制約は、過去10年間にわたって多様な業界や市場のために開発したスケールされた3Dキャプチャソリューションの重要性を強化し、加速させるだけでした。

私たちの空間データプラットフォーム
概要
私たちの技術プラットフォームは、さまざまなデジタルキャプチャデバイスから収集された空間データを使用して、物理的な建物や空間を寸法的に正確でフォトリアルなデジタルツインに変換し、加入者に以前は利用できなかった建物情報や洞察へのアクセスを提供します。
この巨大な市場でほぼ10年間の先駆者として、私たちは、Cortex（私たちの独自のAI駆動ソフトウェアエンジン）によって駆動される業界をリードする3D再構築技術を開発し、スケールしました。Cortexは、機械学習を使用して、建物構造全体のフォトリアルな3D仮想表現を再現します。完成品は、物理的な空間の詳細で動的な複製であり、スマートフォンを含む任意のデバイスのウェブブラウザから探索、分析、カスタマイズできます。大規模な商業ビルを管理する力は、各加入者の手のひらにあり、私たちの高度な技術と空間データ技術スタック全体にわたる画期的な革新によって可能になっています。
私たちの空間データプラットフォームの重要な要素は次のとおりです：
•オフラインの建物をオンラインにする。従来、私たちの顧客は、建物や空間を理解し評価するために対面での現地訪問を行う必要がありました。写真や間取り図は役立つ場合がありますが、これらの2次元（2D）表現は情報が限られており、静的で硬直しているため、各建物や空間の全体的な理解に不可欠なインタラクティブな要素が欠けています。CortexのAI駆動機能により、物理オブジェクトの表現はもはや静的な2D画像に制限されず、物理的な訪問を排除できます。Cortexは、建物や空間をオフラインからオンラインに移動させ、どこからでもリアルタイムでオンデマンドで顧客にアクセスできるようにします。加入者が建物をスキャンした後、私たちの視覚化アルゴリズムは、スキャンを通じてキャプチャされた平面の2D画像から空間の位置と深さを正確に推測し、それらを高忠実度で正確なデジタルツインモデルに変換します。これにより、各デジタルツインがプロフェッショナルグレードの画像品質であることを保証する完全自動化された画像処理パイプラインが作成されます。
•空間データによって駆動される。私たちはデータ駆動型の企業です。空間の各インクリメンタルキャプチャは、私たちの空間データライブラリの豊かさと深さを増します。空間データは、人間が作り出した環境における建物や空間を構成するユニークで特異な詳細を表します。Cortexは、私たちがこれまでに蓄積した数十億のデータポイントの幅を使用して、デジタルツインの3D精度を向上させます。私たちは、加入者がデジタルツイン内のオブジェクトの高さ、位置、その他の特性を特定できるように支援します。私たちの高度なアルゴリズムは、加入者に自信を持って不動産に関する評価や意思決定を行うためのデータに基づく洞察を生成することによって、重要な商業的価値を提供します。たとえば、不動産開発者は特定の窓からの自然熱や日光の量を評価でき、小売業者は各店舗のレイアウトが同じレベルのコードやブランド要件を満たしていることを確認でき、工場は機械のレイアウトが仕様や位置ガイドラインを満たしていることを確認できます。2022年12月31日現在、約920万の空間を管理しており、私たちの空間データライブラリは、構築された世界に関する情報のクリアリングハウスです。
•AIとMLによって駆動される。人工知能と機械学習技術は、空間データを効果的に利用して、動的でリアル、インタラクティブで情報豊富な堅牢な仮想体験を作成し、複数の視点からの表示を許可します。AIとMLは、日常的なスキャンに高価なカメラを必要としなくします。加入者は、スマートフォンのボタンをタップするだけで空間をスキャンできるようになりました。その結果、Matterportはデバイスに依存しないプラットフォームとなり、私たちの構築された世界のデジタル化とインデックス化の使命に向けてより迅速にスケールし、推進しています。
私たちの加入者に対する価値提案は、設計と構築からメンテナンスと運営、プロモーション、販売、リース、保険、修理、復元、セキュリティ、資金調達に至るまで、デジタルビルディングライフサイクル全体にサービスを提供するように設計されています。その結果、私たちは、加入者が建物や空間をオフラインからオンラインに移行する機会を発見するのを助けることで、短期的および長期的な投資収益率を推進するために、収益を成長させるためのユニークな立場にあると信じています。
ユビキタスキャプチャ
Matterportは3D空間キャプチャの標準となりました。私たちの技術プラットフォームは、世界中の加入者が迅速かつ簡単に、正確に建物や空間のインタラクティブで寸法的に正確なデジタルツインをデジタル化、カスタマイズ、管理できるようにします。
Matterportプラットフォームは、すべての加入者のキャプチャニーズに応じて、さまざまなLiDAR、球面、3D、360カメラ、およびスマートフォンと連携するように設計されています。これにより、任意のサイズ、スケール、複雑さの空間を、いつでもどこでもキャプチャする柔軟性が提供されます。
•Matterport Pro3は、プロジェクトの完了を加速するために、以前のバージョンよりも速くプロパティをスキャンする最新の3Dカメラです。Pro3は、屋内および屋外の空間の最高の精度のスキャンを提供し、速度、忠実度、多様性、精度のために設計されています。100メートル離れた場所で3Dデータをキャプチャし、スイープごとに20秒未満で、Pro3の超高速で高精度なLiDARセンサーは数時間動作し、あらゆる条件下で数百万の測定を行います。
•Matterport Pro2は、世界中で数百万の空間を高い忠実度、精度、速度、シンプルさでキャプチャするために使用されている独自の3Dカメラです。50万平方フィート以上の建物をキャプチャでき、多くの住宅、商業、工業、大規模プロパティの選択肢となっています。
•360カメラ。Matterportは、市場で利用可能な360カメラの選択肢をサポートしています。これらの手頃な価格のポケットサイズのデバイスは、高忠実度で精密なキャプチャを提供し、小さな家、コンドミニアム、短期レンタル、アパートなどのキャプチャに適しています。これらのデバイスの球面レンズ画像キャプチャ技術は、Cortexに堅牢で詳細な画像データを提供し、パノラマを業界をリードするデジタルツインに変換します。
•LEICA BLK360。Leicaとの提携を通じて、私たちの3D再構築技術とAI駆動のソフトウェアエンジンであるCortexは、この強力なLiDARカメラをMatterportデジタルツインを作成するための超精密なキャプチャデバイスに変換します。正確な精度が求められる場合、AEC専門家にとっての選択肢です。
•スマートフォンキャプチャ。私たちのキャプチャアプリは、iOSおよびAndroidの両方で商業的に利用可能です。Matterportのスマートフォンキャプチャソリューションは、3Dキャプチャを民主化し、最近のiPhoneデバイスを使用して建物や空間をデジタル化することを誰でも簡単にアクセスできるようにしました。2021年4月には、Androidキャプチャアプリの公式リリースを発表し、Androidユーザーが没入型の3Dで建物や空間を迅速かつ簡単にキャプチャできるようにしました。2022年2月には、Matterport Axisを立ち上げ、スマートフォンを保持し、Matterportキャプチャアプリを使用して、任意の物理空間の3Dデジタルツインをより迅速、正確、一貫してキャプチャできるようにしました。
Cortexと3D再構築（Matterportデジタルツイン）
2022年12月31日現在、約920万の空間を管理し、約280億平方フィートのキャプチャされた空間を表す空間データライブラリを持つ私たちは、先進的なMLおよびDL技術を使用して、キャプチャした空間データをアルゴリズム的に正確な3Dデジタル再現に変換します。このインテリジェントで自動化された3D再構築は、Cortex（私たちのAI駆動のソフトウェアエンジン）によって可能になり、深層学習ニューラルネットワークを含み、私たちの空間データライブラリを使用して、建物や空間がどのようにフロアや部屋に分かれているか、ドアや開口部の位置、どのような部屋が存在するかを理解します。そのため、これらの形状は寸法的に正確に動的でフォトリアルなデジタルツインに編纂され、整列されます。Cortexの他のコンポーネントには、建物や空間内の内容を特定し分類するためのAI駆動のコンピュータビジョン技術や、家具や設備からドア、窓、照明器具、火災抑制スプリンクラー、避難経路に至るまでのすべてを特定しセグメント化するためのオブジェクト認識技術が含まれます。私たちの高度にスケーラブルな人工知能プラットフォームは、加入者がボタンをクリックするだけで強力で強化された建物データと洞察にアクセスできるようにします。

Matterportデジタルツインの背後にある科学：Cortex AIのハイライト
MatterportはCortexで動作します
Cortexは、任意の建物や空間のデジタルツインを作成するための精密な深層学習ニューラルネットワークを含むAI駆動のソフトウェアエンジンです。私たちのPro2およびPro3カメラでキャプチャされた独自の空間データを使用して開発され、Cortexは高い精度と正確性を提供し、日常的なデバイスを使用した3Dキャプチャを可能にします。
一般的なニューラルネットワークは、現実世界の3D再構築に苦労します。Matterport最適化ネットワークは、より正確で堅牢な結果を提供します。生のトレーニングデータ以上のものであり、Matterportのデータセットは、新しいニューラルネットワークアーキテクチャを開発し、数百万の状況におけるユーザー行動や実世界のデータに対して評価することを可能にします。
•深層学習：各空間の詳細なニューラルネットワークデータアーキテクチャを接続し最適化することは、堅牢で非常に正確な3Dデジタルツインを作成するための鍵です。Cortexは、数百万の建物や空間から集約されたMatterportの豊富な空間データに対して各3Dモデルを評価し最適化します。また、世界中の数万の加入者によって提供されたデータの人間による注釈も考慮されます。Cortexの評価能力と3D再構築のデータ駆動の最適化は、さまざまな建物構成、空間、環境にわたって一貫した高精度の結果を生み出します。
•動的3D再構築：2Dビジュアルや静的画像からスケールで正確な3D空間データを作成するには、複数の視点からのフォトリアルで詳細なデータと数百万の空間が必要です。これにより、Cortexのニューラルネットワークと学習能力が改善され、任意の空間の3D再構築が最適化されます。Cortexの機能と、私たちの3Dキャプチャ技術におけるリアルタイム空間整列アルゴリズムを組み合わせることで、進行中の作業の直感的な「プレビュー」を作成し、加入者がコンテンツをインタラクティブかつリアルタイムで操作できるようにします。
•コンピュータビジョン：Cortexは、デジタルツインの価値を高めるための強力な機能のスイートを提供します。これには、部屋や部屋内のオブジェクトの自動測定、自動2Dから3Dへの高解像度フォトギャラリーの作成、プライバシー保護のための自動顔ぼかし、カスタムビデオ、ウォークスルー、自動部屋ラベリング、オブジェクト認識が含まれます。
•高度な画像処理：Matterportの計算写真アルゴリズムは、各デジタルツインがプロフェッショナルグレードの画像品質であることを保証するための完全自動化された画像処理パイプラインを作成します。私たちの特許技術により、3Dキャプチャはボタンを押すだけで簡単になります。Matterportのソフトウェアと技術は、ホワイトバランスやカメラ特有の色補正、高ダイナミックレンジトーンマッピング、ノイズ除去、霞除去、シャープ化、彩度、その他の調整を含む残りのステップを管理します。
空間データとAI駆動の洞察
すべてのMatterportデジタルツインには、建物、部屋、または物理空間に関する広範な情報が含まれています。このデータは、私たちのAI駆動のCortexエンジンを使用しています。Matterportデジタルツイン自体に加えて、私たちの空間データは、精密な建物のジオメトリと構造の詳細、建物の内容、備品、状態、空間内の多くの視点からの高解像度の画像とフォトリアルな詳細で構成されています。Cortexは、私たちが「深い空間インデックス」と呼ぶ技術を使用しています。深い空間インデックスは、人工知能、コンピュータビジョン、深層学習を使用して、各空間、その構造、および内容に関する重要な詳細を正確かつ忠実に特定し伝達します。私たちは、Matterportの加入者が任意の建物のための相互運用可能なデジタル記録システムを活用できるようにする堅牢な空間データ標準を作成しました。
デジタルツインの構築を通じて加入者に対して高度にインタラクティブなデジタル体験を提供するだけでなく、私たちはすべての加入者に対して2つの質問を自問します：（1）彼らの建物や物理空間にとって重要なことは何か、（2）この空間に対してどのような学びや洞察を提供できるか。私たちのAI駆動のCortexエンジンは、私たちの空間データライブラリを使用して、集約された不動産トレンドや運用および評価の洞察を提供することによって、これらの質問に答えるのを助けます。さらに、Matterportプラットフォームエコシステムが拡大し続けるにつれて、私たちの加入者、パートナー、他のサードパーティ開発者は、私たちの豊富な空間データ層から収穫できる洞察の幅と深さをさらに広げるために独自のツールを持ち込むことができます。
拡張可能なプラットフォームエコシステム
Matterportは、2022年12月31日現在、約920万の空間を管理し、約280億平方フィートをキャプチャした世界で最も大きく、最も正確な空間データライブラリを提供しています。私たちの空間データプラットフォームの多様性と広範なエンタープライズソフトウェア開発キットおよびアプリケーションプログラミングインターフェース（API）は、地理や垂直市場によってMatterportの価値提案を拡張する堅牢なグローバルエコシステムのチャネルとパートナーを開発することを可能にしました。私たちは、ターゲット市場全体で統合されたMatterportソリューションを促進するために、パートナーやその加入者との広範なワークフロー統合を展開し続けるつもりです。また、私たちは、簡単に展開でき、簡単にアクセスできるMatterportソフトウェアアドオンで空間データプラットフォームの力を拡張するためのサードパーティソフトウェアマーケットプレイスを開発しています。このマーケットプレイスは、開発者が新しいアプリケーションや空間データマイニングツールを構築し、Matterportの3D体験を強化し、コア提供物を補完する新しい生産性および不動産管理ツールを作成できるようにします。サードパーティ開発者によって作成されたこれらの付加価値機能は、スケーラブルな新しい収益源を可能にし、Matterportはオンラインマーケットプレイスを通じて加入者に展開される各アドオンからのサブスクリプションおよびサービス収益を共有します。私たちのプラットフォームエコシステムのネットワーク効果は、私たちのビジネスの成長に寄与し、加入者の粘着性とユーザーエンゲージメントを高めることによって将来の成長を支えると信じています。
Matterportのアドオンや拡張の例には、次のものがあります：
•アドオン：Encircle（迅速な請求処理のための使いやすいフィールドドキュメンテーションツール）；WP Matterport Shortcode（Matterportを迅速かつ簡単に埋め込むことを可能にする無料のWordpressプラグイン）、WP3D Models（WordPress + Matterport統合プラグイン）；Rela（リスティングのためのオールインワンマーケティングソリューション）；CAPTUR3D（Matterportデジタルツインに価値を拡張するオールインワンコンテンツ管理システム）；Private Model Emded（企業が追加のユーザーライセンスを必要とせずに、企業ネットワーク上の大規模な従業員グループとデジタルツインをプライベートに共有できる機能）；Views（グループや大規模な組織が異なるチームで異なるタスクを管理するための別々の権限ベースのワークフローを作成できる新しい作業グループコラボレーションフレームワーク）；およびGuided Tours and Tags（訪問者の興味に合わせた商業または住宅空間の指向されたバーチャルツアーを作成するためのツール）。私たちは、企業顧客がIoTデータを視覚的に没入型で寸法的に正確なMatterportデジタルツインにシームレスに接続できるようにするために、Amazon Web Services（AWS）IoT TwinMakerとのプライベートベータ統合を発表しました。
•サービス：Matterport ADA準拠デジタルツイン（アメリカ障害者法に準拠したデジタルツインを提供するソリューション）およびエンタープライズクラウドソフトウェアプラットフォーム（任意のサイズや形状の建物や空間のデジタルツインを作成、公開、管理するための再構築されたクラウドソフトウェアプラットフォーム）。
私たちの競争力の強み
私たちは、市場リーダーシップを成長させるための競争力の強みがいくつかあると信じています。私たちの競争力の強みには、次のものが含まれます：
•Matterportプラットフォームの幅と深さ。私たちのコアの強みは、カスタマイズなしでキャプチャから処理、業界全体にわたる広範なリーチを持つオールインワンの空間データプラットフォームです。さまざまなエンタープライズシステムとシームレスに統合できる能力を持つ私たちのプラットフォームは、不動産、AEC、旅行・ホスピタリティ、修理・保険、工業・施設などの多様なエンド市場にわたって、プロパティライフサイクル全体に価値を提供します。2022年12月31日現在、私たちのグローバルなリーチは170カ国以上の加入者に拡大し、フォーチュン1000企業の24%以上を含んでいます。
•市場リーダーシップと先駆者の利点。Matterportは、ほぼ10年前に構築された世界のデジタル化とデータ化のカテゴリを定義し、このカテゴリのグローバルリーダーとなりました。2022年12月31日現在、私たちは701,000人以上の加入者をプラットフォーム上に持ち、約920万の空間を管理しています。私たちのリーダーシップは、デジタルツイン作成における先駆者であったことに主に起因しています。先駆者の利点の結果、私たちは、リーダーシップの地位を強化し続ける豊かで深い空間データライブラリを蓄積しました。
•重要なネットワーク効果。新しいキャプチャやデータがプラットフォームに追加されるたびに、データセットの豊かさと管理されている空間からの洞察の深さが増します。さらに、データを洞察に変える能力と、加入者による新しいデータキャプチャからの追加データの組み合わせにより、Matterportは加入者向けの機能を開発できます。私たちは、構築された世界のための空間データライブラリを構築する先駆者であり、データから洞察を収集し導き出すリーダーシップは、ますます強化され、その洞察の関連性が新しい加入者を引き寄せます。
•貴重な不動産洞察のための原材料としての巨大な空間データライブラリ。私たちの空間データライブラリの規模は、加入者のための洞察を導き出す上で重要な利点です。私たちの空間データライブラリは、Cortexの重要な真実の基盤として機能し、Matterportが低コストのデジタルおよびスマートフォンカメラを使用して強力な3Dデジタルツインを作成できるようにします。2022年12月31日現在、私たちのデータは約920万の空間から管理され、約280億平方フィートがキャプチャされています。その結果、私たちは不動産の洞察と分析を新たなレベルに引き上げ、さまざまな業界の加入者に利益をもたらしています。たとえば、施設管理者は建物のレイアウトを作成するのに必要な時間を大幅に短縮し、現地調査や実測モデリングのコストを大幅に削減します。AECの加入者は、各実測空間の分析を使用して文書化を効率化し、簡単にコラボレーションできます。
•グローバルなリーチとスケール。私たちは、AI駆動の空間データプラットフォームを世界中に拡大し続けることに注力しています。私たちは、北米、ヨーロッパ、アジアにおいて重要な存在感を持ち、各地域においてリーダーシップチームと市場投入インフラを持っています。私たちはロンドン、シンガポール、アメリカ各地にオフィスを持ち、国際的な拡大を加速しています。2022年12月31日現在、私たちは170カ国以上で701,000人以上の加入者を持っています。私たちの空間データプラットフォームの地理に依存しない性質は、国際的に成長し続ける上で重要な利点であると信じています。
•10年間の研究開発と革新を支える広範な特許ポートフォリオ。2022年12月31日現在、私たちは54件の特許が発行され、37件の特許出願が保留中です。私たちの成功は、革新に対するほぼ10年間の焦点に基づいています。革新はMatterportの中心にあり、私たちは市場のリーディングポジションをさらに強化するために研究開発への投資を優先し続けます。
•優れたキャプチャ技術。Matterportのキャプチャ技術プラットフォームは、建物や空間のMatterportデジタルツインを作成するために必要なさまざまなキャプチャデバイスをサポートするソフトウェアフレームワークです。
これには、LiDARカメラ、360カメラ、スマートフォン、Matterport Axis、Matterport Pro2およびPro3カメラのサポートが含まれます。Pro2カメラは、私たちの空間データの利点の基盤となり、Matterport対応のサードパーティキャプチャデバイスの配列でその利点を拡大しました。2022年8月には、Pro3カメラの発売を開始し、業界をリードするデジタルツインクラウドプラットフォームに大規模なアップデートを行いました。Matterport Pro3カメラは、高速な起動時間、交換可能なバッテリー、軽量設計を備えた高度な3Dキャプチャデバイスです。Pro3カメラは、屋内外の両方で機能し、速度、忠実度、多様性、精度のために設計されています。Pro2カメラとともに、私たちはPro3カメラの将来の販売が私たちのソリューションの採用を引き続き促進すると期待しています。Matterportは3Dキャプチャ体験を民主化し、高忠実度で高精度な3Dデジタルツインを、あらゆる建物タイプや不動産ライフサイクルのニーズに対して容易に利用できるようにしています。他の3Dキャプチャソリューションプロバイダーは存在しますが、真の寸法的に正確な3D結果を生成できるものは非常に少なく、フォトリアルな3Dで最終製品を自動的に作成できるものはさらに少ないです。この広範なキャプチャ技術の提供は、スマートフォンや360カメラでキャプチャされた写真から正確なデジタルツインを自動的に生成するために、AI駆動のCortexエンジンをトレーニングするために利用可能な豊富な空間データライブラリがなければ実現できません。
</document>""" 

# Claudeの応答を出力
print(get_completion(PROMPT))


In [ ]:
# プロンプト
PROMPT = """<question>2020年5月31日の正確な日付におけるMatterportの加入者数は何人でしたか？</question>
以下の文書をお読みください。その後、<scratchpad>タグ内に文書から最も関連性の高い引用を引き出し、それがユーザーの質問に対する答えとなるか、十分な詳細が欠けているかを考慮してください。次に、<answer>タグ内に簡潔な数値回答を書いてください。

<document>
Matterport SEC filing 10-K 2023
項目1. ビジネス
私たちの会社
Matterportは、構築された世界のデジタル化とデータ化をリードしています。私たちは、構築された世界のデジタル変革が、人々が建物や周囲の物理的空間と相互作用する方法を根本的に変えると信じています。
2011年の設立以来、Matterportの先駆的な技術は、建物、空間、場所をオンラインでデジタル化、アクセス、管理するための基準を設定してきました。私たちのプラットフォームの革新的なソフトウェア、空間データ駆動のデータサイエンス、3Dキャプチャ技術は、世界最大の資産クラスである建物や物理的空間を長年にわたりオフラインで未活用のままにしていた障壁を打破しました。私たちは、構築された世界のデジタル化とデータ化が、重要な運用効率と不動産価値を解放し続けると信じており、Matterportがこの巨大なグローバル変革をリードするプラットフォームであると考えています。
世界は急速にオフラインからオンラインへと移行しています。デジタル変革は、今日のすべてのビジネスや業界に強力で持続的な影響を与えています。国際データ公社（IDC）によると、2020年から2023年の間にデジタル変革に対して6.8兆ドル以上の直接投資が行われるとされ、2026年にはグローバルなデジタル変革支出が3.4兆ドルに達し、5年間の年平均成長率（CAGR）は16.3%になると予測されています。また、デジタルツインへの投資は、5年間のCAGRが35.2%になると予想されています。この世俗的なシフトに伴い、物理的なものからデジタルへの移行に対する需要も高まっています。それにもかかわらず、建物や空間の大多数は依然としてオフラインで未デジタル化のままです。2021年時点でサヴィルズによって推定されたグローバルな建物ストックは327兆ドルの総不動産価値があり、今日でも大部分がオフラインのままです。私たちは、デジタル変革によって浸透しているのは0.1%未満であると推定しています。
Matterportは、構築された世界のデジタル化の必要性と、建物や空間の理解を促進するための空間データの力を認識した最初の企業の一つでした。過去には、技術が物理的な地図をデータが豊富なデジタル地図や位置情報サービスに進化させました。Matterportは現在、建物をデジタル化し、キャプチャしたすべての空間の理解と潜在能力を大幅に向上させるデータが豊富な環境を作り出しています。私たちがボタンを押すだけで、ある都市から別の都市への最速ルートを瞬時に学んだり、最寄りのコーヒーショップを見つけたりできるように、Matterportの建物に関する空間データは、世界中の不動産や空間に関する豊富な洞察と学びを解放します。さらに、今日の地理空間マッピングプラットフォームが業界にマッピングデータを開放し、ライドシェアリング、eコマース、フードデリバリーマーケットプレイス、さらには短期レンタルやホームシェアリングなどの新しいビジネスモデルを創出しているのと同様に、Matterportの構造化された空間データへのオープンアクセスは、ホスピタリティ、施設管理、保険、建設、不動産、小売などの新しい機会とビジネスモデルを可能にしています。
私たちは、構築された世界のデジタル化に対する総アドレス可能市場機会が2400億ドルを超え、市場がスケールで成熟するにつれて1兆ドルに達する可能性があると信じています。これは、世界中の40億以上の不動産と200億の空間のグローバルな建物ストックに関する私たちの分析、モデリング、理解に基づいています。人工知能（AI）、機械学習（ML）、深層学習（DL）技術の助けを借りて、私たちは、強力な空間データ駆動の不動産洞察と分析からの追加の収益化機会を考慮すると、構築された世界のデジタル化とデータ化のための総アドレス可能市場が1兆ドルを超えると信じています。

私たちの空間データプラットフォームとデジタルツインのキャプチャは、多様な業界やユースケースに価値を提供します。大手小売業者は、数千の店舗をリモートで管理でき、不動産会社は数百の物件と数千の訪問者のためにバーチャルオープンハウスを提供でき、プロパティ開発者は建設プロセス全体をより詳細かつ迅速に監視でき、保険会社は効率的かつ正確に請求やアンダーライティングの評価を文書化し評価できます。Matterportは、世界中のほぼすべてのサイズ、形状、場所の不動産に関して、加入者にとって重要なデジタル体験、ツール、情報を提供します。
私たちは、ほぼ10年間、空間データプラットフォームを成長させ、最も詳細で正確かつデータが豊富なデジタルツインを作成するための能力を拡大してきました。さらに、私たちの3D再構築プロセスは完全に自動化されており、私たちのソリューションは、世界中のあらゆるタイプ、形状、サイズの数百万の建物や空間に対して同じ精度でスケールすることができます。私たちのサービスの普遍的な適用性は、Matterportに多様な垂直市場や地理にわたる大規模なスケールとリーチを提供します。2022年12月31日現在、私たちの加入者数は、2021年12月31日現在の503,000人から約39%増加し、701,000人を超え、私たちのデジタルツインは170カ国以上に広がっています。私たちは、複数の業界で280億平方フィート以上の空間をデジタル化しており、市場全体に対して重要なスケールと成長を示しています。

私たちが世界中の建物をデータに変換し続ける中で、私たちは空間データプラットフォームを拡張し、加入者のために物件の計画、開発、管理、インテリジェンスをさらに変革し、構築された世界のための事実上の建物およびビジネスインテリジェンスエンジンとなることを目指しています。私たちは、不動産、建築、エンジニアリングおよび建設（AEC）、小売、保険、政府を含む業界全体で、企業、ビジネス、機関に対する空間データとそれに伴う洞察の需要が急速に増加し続けると信じています。
私たちは、デジタル化とデータ化が、この巨大なカテゴリと資産クラス全体にわたる成長のための素晴らしいグリーンフィールドの機会を表していると信じています。設計と開発の初期段階から、マーケティング、運営、保険、建物の修理とメンテナンスに至るまで、私たちのプラットフォームのソフトウェアと技術は、加入者にコスト削減、収益増加、建物や空間の最適な管理を推進するための重要なツールと洞察を提供します。私たちは、構築された世界における未実現の利用と運用効率に数千億ドルが解放される可能性があると信じています。私たちのプラットフォームとデータソリューションは、業界や建物カテゴリ全体に普遍的に適用可能であり、Matterportにとっては、この大規模な市場機会全体に対処し、世界で最も大きな資産クラスの価値を高めるという重要な利点を提供します。
加入者に価値を提供する実績を持つ私たちの提供物には、ソフトウェアサブスクリプション、データライセンス、サービス、製品ハードウェアが含まれます。2022年12月31日現在、私たちの加入者数には、フォーチュン1000企業の24%以上が含まれており、上位10社からの総収益の10%未満が生成されています。2025年までに、私たちの収益の80%以上がソフトウェアサブスクリプションとデータライセンスソリューションから来ると予想しています。私たちの革新的な3Dキャプチャ製品であるPro2およびPro3カメラは、3D建物および不動産ビジュアライゼーションエコシステムの形成に重要な役割を果たしています。Pro2およびPro3カメラは、私たちのソリューションの採用を促進し、Cortex、私たちの独自のAIソフトウェアエンジンがデジタルツイン作成の先駆的なエンジンとなるために必要なユニークで高品質なスケールデータセットを生成しました。このデータの利点は、最初にPro2カメラによって促進され、私たちはキャプチャデバイスに依存しないプラットフォームを開発し、業界や地理にわたって加入者のために新しい建物や不動産の洞察を生成できるようにしました。
私たちは最近、急速な成長を経験しました。私たちの加入者数は、2018年12月31日から2022年12月31日までに約49倍に増加しました。私たちの収益は、2022年12月31日までの年度で約22%増加し、1億3610万ドルに達しました。2021年12月31日までの年度では約1億1120万ドルでした。私たちの粗利益は、2022年12月31日までの年度で810万ドル、または14%減少し、5180万ドルになりました。これは、2022会計年度におけるグローバルなサプライチェーンの制約による特定の混乱的および追加コストに起因しています。既存の加入者から生成されるサブスクリプション収益を保持し、成長させる能力は、私たちのビジネスの健全性と将来の成長の見通しを測る重要な指標です。この分野でのパフォーマンスを追跡するために、私たちは同じ顧客セットからのネットドル拡張率を測定しています。2022年12月31日までの3か月間のネットドル拡張率は103%であり、私たちのプラットフォームの粘着性と成長の可能性を示しています。
私たちの業界と市場機会
今日、建物や空間の大多数は未デジタル化のままです。私たちは、現在のサービス可能なアドレス市場が、主に不動産および旅行・ホスピタリティセクターから、世界中で約13億の空間を含むと推定しています。2022年12月31日現在、約920万の空間を管理しており、私たちはグローバルな建物ストックに浸透し続け、住宅および商業不動産、施設管理、小売、AEC、保険および修理、旅行およびホスピタリティを含むさまざまなエンド市場にわたって足跡を拡大しています。私たちは、総アドレス可能市場が世界中で40億以上の建物と200億の空間を超え、2400億ドル以上の市場機会を生み出すと推定しています。Matterportのユニークな空間データライブラリと不動産データサービスが成長し続けるにつれて、この機会は、建物ストックの規模と世界中の建物に対する未開発の価値創造に基づいて、1兆ドルを超える可能性があります。COVID-19パンデミックによって生じた制約は、過去10年間にわたって多様な業界や市場のために開発したスケールされた3Dキャプチャソリューションの重要性を強化し加速させるだけでした。

私たちの空間データプラットフォーム
概要
私たちの技術プラットフォームは、さまざまなデジタルキャプチャデバイスから収集された空間データを使用して、物理的な建物や空間を寸法的に正確でフォトリアリスティックなデジタルツインに変換し、加入者に以前は利用できなかった建物情報や洞察へのアクセスを提供します。
この巨大な市場でほぼ10年間の先駆者として、私たちは、Cortexという独自のAI駆動ソフトウェアエンジンによって駆動される業界をリードする3D再構築技術を開発し、スケールしました。Cortexは、機械学習を使用して、建物構造全体のフォトリアリスティックな3D仮想表現を再現します。完成品は、物理的な空間の詳細で動的な複製であり、スマートフォンを含む任意のデバイスのウェブブラウザから探索、分析、カスタマイズできます。大規模な商業ビルを管理する力は、各加入者の手のひらにあり、私たちの先進的な技術と空間データ技術スタック全体にわたる画期的な革新によって可能になっています。
私たちの空間データプラットフォームの重要な要素は以下の通りです：
•オフラインの建物をオンラインにする。従来、私たちの顧客は、建物や空間を理解し評価するために対面での現地訪問を行う必要がありました。写真や間取り図は役立つことがありますが、これらの2次元（2D）表現は情報が限られており、静的で硬直しているため、各建物や空間の全体的な理解に重要なインタラクティブな要素が欠けています。CortexのAI駆動機能により、物理的なオブジェクトの表現はもはや静的な2D画像に制限されず、物理的な訪問を排除できます。Cortexは、建物や空間をオフラインからオンラインに移行し、顧客がどこからでもリアルタイムでオンデマンドでアクセスできるようにします。加入者が建物をスキャンした後、私たちの視覚化アルゴリズムは、スキャンを通じてキャプチャされた平面の2D画像から空間の位置と深さを正確に推測し、それらを高忠実度で正確なデジタルツインモデルに変換します。これにより、各デジタルツインがプロフェッショナルグレードの画像品質であることを保証する完全自動化された画像処理パイプラインが作成されます。
•空間データによって駆動される。私たちはデータ駆動型の会社です。空間の各インクリメンタルキャプチャは、私たちの空間データライブラリの豊かさと深さを増します。空間データは、人間が作り出した環境における建物や空間を構成するユニークで特異な詳細を表します。Cortexは、私たちがこれまでに蓄積した数十億のデータポイントの幅を利用して、デジタルツインの3D精度を向上させます。私たちは、加入者がデジタルツイン内のオブジェクトの高さ、位置、その他の特性を特定できるように支援します。私たちの高度なアルゴリズムは、加入者に商業的価値を提供し、彼らが自分の不動産について自信を持って評価や意思決定を行えるようにするデータベースの洞察を生成します。たとえば、プロパティ開発者は特定の窓からの自然熱や日光の量を評価し、小売業者は各店舗のレイアウトが同じレベルのコードやブランド要件を満たしていることを確認し、工場は機械のレイアウトが仕様や位置ガイドラインを満たしていることを確認できます。2022年12月31日現在、約920万の空間を管理しており、私たちの空間データライブラリは構築された世界に関する情報のクリアリングハウスです。
•AIとMLによって駆動される。人工知能と機械学習技術は、空間データを効果的に利用して、動的でリアルなインタラクティブな体験を作成し、情報を提供し、複数の視点からの表示を許可します。AIとMLは、日常的なスキャンに高価なカメラを必要としなくします。加入者は、スマートフォンのボタンをタップするだけで空間をスキャンできるようになりました。その結果、Matterportはデバイスに依存しないプラットフォームとなり、私たちの構築された世界のデジタル化とインデックス化の使命に向けてより迅速にスケールし、推進しています。
私たちの加入者に対する価値提案は、設計と構築からメンテナンスと運営、プロモーション、販売、リース、保険、修理、復元、セキュリティ、資金調達に至るまで、デジタル建物ライフサイクル全体にサービスを提供するように設計されています。その結果、私たちは、加入者が自分の建物や空間をオフラインからオンラインに移行する機会を発見するのを助けることで、短期的および長期的な投資収益率を推進する機会を見出しながら、収益を成長させるためのユニークな立場にあると信じています。
ユビキタスキャプチャ
Matterportは3D空間キャプチャの標準となりました。私たちの技術プラットフォームは、世界中の加入者が自分の建物や空間のインタラクティブで寸法的に正確なデジタルツインを迅速かつ簡単に、正確にデジタル化、カスタマイズ、管理できるようにします。
Matterportプラットフォームは、さまざまなLiDAR、球面、3D、360カメラ、スマートフォンと連携して、すべての加入者のキャプチャニーズに対応するように設計されています。これにより、任意のサイズ、スケール、複雑さの空間を、いつでもどこでもキャプチャする柔軟性が提供されます。
•Matterport Pro3は、プロジェクトの完了を加速するために、以前のバージョンよりも速くプロパティをスキャンする最新の3Dカメラです。Pro3は、屋内および屋外の空間の最高の精度のスキャンを提供し、速度、忠実度、多様性、精度のために設計されています。100メートル離れた場所で3Dデータをキャプチャし、スイープごとに20秒未満で、Pro3の超高速で高精度なLiDARセンサーは数時間動作し、あらゆる条件下で数百万の測定を行います。
•Matterport Pro2は、世界中で数百万の空間を高い忠実度、精度、速度、シンプルさでキャプチャするために使用されている独自の3Dカメラです。50万平方フィートを超える建物をキャプチャでき、多くの住宅、商業、工業、大規模プロパティのための選択されたカメラとなっています。
•360カメラ。Matterportは、市場で利用可能な360カメラの選択をサポートしています。これらの手頃な価格のポケットサイズのデバイスは、高忠実度で精密なキャプチャを提供し、小さな家、コンドミニアム、短期レンタル、アパートなどのキャプチャに適しています。これらのデバイスの球面レンズ画像キャプチャ技術は、Cortexに堅牢で詳細な画像データを提供し、パノラマを業界をリードするデジタルツインに変換します。
•LEICA BLK360。ライカとの提携を通じて、私たちの3D再構築技術とAI駆動のソフトウェアエンジンであるCortexは、この強力なLiDARカメラをMatterportデジタルツインを作成するための超精密なキャプチャデバイスに変換します。正確な精度が求められる場合、AEC専門家にとっての選択肢です。
•スマートフォンキャプチャ。私たちのキャプチャアプリは、iOSおよびAndroidの両方で商業的に利用可能です。Matterportのスマートフォンキャプチャソリューションは、3Dキャプチャを民主化し、最近のiPhoneデバイスを使用して誰でも建物や空間をデジタル化できるようにしました。2020年5月にMatterport for iPhoneの初回導入以来、2021年4月にはAndroid Captureアプリの公式リリースを発表し、Androidユーザーが没入型の3Dで建物や空間を迅速かつ簡単にキャプチャできるようにしました。2022年2月には、Matterport Axisを立ち上げ、スマートフォンを保持するモーターマウントを使用して、Matterport Captureアプリで任意の物理空間の3Dデジタルツインをより速く、正確に、一貫してキャプチャできるようにしました。
Cortexと3D再構築（Matterportデジタルツイン）
2022年12月31日現在、約920万の空間を管理し、約280億平方フィートのキャプチャされた空間を表す空間データライブラリを持つ私たちは、先進的なMLおよびDL技術を使用して、キャプチャした空間データをアルゴリズム的に正確な3Dデジタル再現に変換します。このインテリジェントで自動化された3D再構築は、Cortexによって可能になり、私たちの空間データライブラリを使用して、建物や空間がどのようにフロアや部屋に分かれているか、ドアや開口部の位置、どのような部屋が存在するかを理解します。その形状は、寸法的な正確さで動的でフォトリアリスティックなデジタルツインに編纂され、整列されます。Cortexの他のコンポーネントには、建物や空間内の内容を特定し分類するためのAI駆動のコンピュータビジョン技術や、家具や設備からドア、窓、照明器具、火災抑制スプリンクラー、非常口までを特定しセグメント化するための物体認識技術が含まれます。私たちの高度にスケーラブルな人工知能プラットフォームは、加入者がボタンをクリックするだけで強力で強化された建物データと洞察にアクセスできるようにします。

Matterportデジタルツインの背後にある科学：Cortex AIのハイライト
MatterportはCortexで動作します
Cortexは、任意の建物や空間のデジタルツインを作成するための精密な深層学習ニューラルネットワークを含むAI駆動のソフトウェアエンジンです。Pro2およびPro3カメラでキャプチャされた独自の空間データを使用して開発され、Cortexは高い精度と正確性を提供し、日常的なデバイスを使用した3Dキャプチャを可能にします。
一般的なニューラルネットワークは、現実世界の3D再構築に苦労します。Matterport最適化ネットワークは、より正確で堅牢な結果を提供します。生のトレーニングデータ以上のものであり、Matterportのデータセットは、新しいニューラルネットワークアーキテクチャを開発し、数百万の状況におけるユーザー行動や実世界のデータに対して評価することを可能にします。
•深層学習：各空間の詳細なニューラルネットワークデータアーキテクチャを接続し最適化することは、堅牢で非常に正確な3Dデジタルツインを作成するための鍵です。Cortexは、数百万の建物や空間から集約されたMatterportの豊富な空間データに対して各3Dモデルを評価し最適化します。Cortexの評価能力と3D再構築のデータ駆動の最適化は、さまざまな建物構成、空間、環境にわたって一貫した高精度の結果を生み出します。
•動的3D再構築：2Dビジュアルや静的画像からスケールで正確な3D空間データを作成するには、複数の視点からのフォトリアリスティックで詳細なデータと数百万の空間が必要です。これにより、Cortexのニューラルネットワークと学習能力が改善され、任意の空間の3D再構築が最適化されます。Cortexの機能と、私たちの3Dキャプチャ技術におけるリアルタイム空間整列アルゴリズムを組み合わせることで、進行中の作業の直感的な「プレビュー」を作成し、加入者がコンテンツをインタラクティブかつリアルタイムで操作できるようにします。
•コンピュータビジョン：Cortexは、デジタルツインの価値を高めるための強力な機能のスイートを提供します。これには、部屋や部屋内のオブジェクトの自動測定、自動2Dから3Dへの高解像度フォトギャラリー作成、プライバシー保護のための自動顔ぼかし、カスタムビデオ、ウォークスルー、自動部屋ラベリング、物体認識が含まれます。
•高度な画像処理：Matterportの計算写真アルゴリズムは、各デジタルツインがプロフェッショナルグレードの画像品質であることを保証するための完全自動化された画像処理パイプラインを作成します。私たちの特許技術により、3Dキャプチャはボタンを押すだけで簡単になります。Matterportのソフトウェアと技術は、ホワイトバランスやカメラ特有の色補正、高ダイナミックレンジトーンマッピング、ノイズ除去、霞除去、シャープ化、彩度、その他の調整を含む残りのステップを管理します。

空間データとAI駆動の洞察
すべてのMatterportデジタルツインには、建物、部屋、または物理空間に関する広範な情報が含まれています。このデータは、私たちのAI駆動のCortexエンジンを使用しています。Matterportデジタルツイン自体に加えて、私たちの空間データは、精密な建物のジオメトリと構造的詳細、建物の内容、設備、状態、空間内の多くの視点からの高解像度画像とフォトリアリスティックな詳細で構成されています。Cortexは、私たちが「深い空間インデックス」と呼ぶ技術を採用しています。深い空間インデックスは、人工知能、コンピュータビジョン、深層学習を使用して、各空間、その構造、および内容に関する重要な詳細を正確かつ忠実に特定し伝達します。私たちは、Matterportの加入者が任意の建物のための相互運用可能なデジタル記録システムを活用できるようにする堅牢な空間データ標準を作成しました。
デジタルツインの構築を通じて加入者に高度にインタラクティブなデジタル体験を提供するだけでなく、私たちはすべての加入者に対して2つの質問を自問します：（1）彼らの建物や物理空間にとって重要なことは何か、（2）この空間に対してどのような学びや洞察を提供できるか？私たちのAI駆動のCortexエンジンは、私たちの空間データライブラリを使用して、集約されたプロパティトレンドや運用および評価の洞察を提供することで、これらの質問に答えるのを助けます。さらに、Matterportプラットフォームエコシステムが拡大し続けるにつれて、私たちの加入者、パートナー、他のサードパーティ開発者は、自分たちのツールを持ち込んで、私たちの豊かな空間データ層から収穫できる洞察の幅と深さをさらに広げることができます。
拡張可能なプラットフォームエコシステム
Matterportは、2022年12月31日現在、約920万の空間を管理し、約280億平方フィートをキャプチャした世界で最も大きく、最も正確な空間データライブラリを提供しています。私たちの空間データプラットフォームの多様性と広範なエンタープライズソフトウェア開発キットおよびアプリケーションプログラミングインターフェース（API）は、Matterportの価値提案を地理的および垂直市場で拡張する堅牢なグローバルエコシステムを開発することを可能にしました。私たちは、ターゲット市場全体で統合されたMatterportソリューションを促進するために、パートナーやその加入者との広範なワークフロー統合を展開し続けるつもりです。また、私たちは、簡単に展開でき、アクセスしやすいMatterportソフトウェアアドオンで空間データプラットフォームの力を拡張するためのサードパーティソフトウェアマーケットプレイスを開発しています。このマーケットプレイスは、開発者が新しいアプリケーションや空間データマイニングツールを構築し、Matterportの3D体験を強化し、コア提供物を補完する新しい生産性およびプロパティ管理ツールを作成できるようにします。サードパーティ開発者によって作成されたこれらの付加価値機能は、スケーラブルな新しい収益源を可能にし、Matterportはオンラインマーケットプレイスを通じて加入者に展開される各アドオンからのサブスクリプションおよびサービス収益を共有します。私たちのプラットフォームエコシステムのネットワーク効果は、私たちのビジネスの成長に寄与し、加入者の粘着性とユーザーエンゲージメントを高めることで、将来の成長を支えると信じています。
Matterportのアドオンや拡張の例には以下が含まれます：
•アドオン：Encircle（迅速な請求処理のための使いやすいフィールドドキュメンテーションツール）；WP Matterport Shortcode（Matterportを迅速かつ簡単に埋め込むことを可能にする無料のWordpressプラグイン）、WP3D Models（WordPress + Matterport統合プラグイン）；Rela（リスティングのためのオールインワンマーケティングソリューション）；CAPTUR3D（Matterportデジタルツインに価値を拡張するオールインワンコンテンツ管理システム）；Private Model Emded（企業が追加のユーザーライセンスを必要とせずに、企業ネットワーク上の大規模な従業員グループとデジタルツインをプライベートに共有できる機能）；Views（グループや大規模な組織が異なるチームで異なるタスクを管理するための別々の権限ベースのワークフローを作成できる新しい作業グループコラボレーションフレームワーク）；およびGuided Tours and Tags（訪問者の興味に合わせた商業または住宅空間の指向されたバーチャルツアーを作成するツール）。私たちは、企業顧客がIoTデータを視覚的に没入型で寸法的に正確なMatterportデジタルツインにシームレスに接続できるようにするために、Amazon Web Services（AWS）IoT TwinMakerとのプライベートベータ統合を発表しました。
•サービス：Matterport ADA準拠デジタルツイン（アメリカ障害者法に準拠したデジタルツインを提供するソリューション）およびエンタープライズクラウドソフトウェアプラットフォーム（任意のサイズや形状の建物や空間のデジタルツインを作成、公開、管理するための再構築されたクラウドソフトウェアプラットフォーム）。
私たちの競争力の強み
私たちは、市場リーダーシップを成長させるためのいくつかの競争力の強みを持っていると信じています。私たちの競争力の強みには以下が含まれます：
•Matterportプラットフォームの幅と深さ。私たちのコアの強みは、カスタマイズなしでキャプチャから処理、業界全体にわたる広範なリーチを持つオールインワンの空間データプラットフォームです。さまざまなエンタープライズシステムとシームレスに統合できる能力を持つ私たちのプラットフォームは、不動産、AEC、旅行・ホスピタリティ、修理・保険、工業・施設などの多様なエンド市場にわたって、プロパティライフサイクル全体に価値を提供します。2022年12月31日現在、私たちのグローバルなリーチは170カ国以上の加入者に拡大し、フォーチュン1000企業の24%以上を含んでいます。
•市場リーダーシップと先行者利益。Matterportは、ほぼ10年前に構築された世界のデジタル化とデータ化のカテゴリを定義し、このカテゴリのグローバルリーダーとなりました。2022年12月31日現在、私たちは701,000人以上の加入者をプラットフォーム上に持ち、約920万の空間を管理しています。私たちのリーダーシップは、デジタルツイン作成における先行者利益によって主に推進されています。その結果、私たちは、深く豊かな空間データライブラリを蓄積し続けており、それが私たちのリーダーシップポジションを強化しています。
•重要なネットワーク効果。新しいキャプチャやデータがプラットフォームに追加されるたびに、データセットの豊かさと管理されている空間からの洞察の深さが増します。さらに、データを洞察に変える能力と、加入者による新しいデータキャプチャからのインクリメンタルデータの組み合わせにより、Matterportは加入者向けの機能を開発できます。私たちは、構築された世界のための空間データライブラリを構築する先行者であり、データから洞察を収集し導き出すリーダーシップは、ますます強化され、その洞察の関連性が新しい加入者を引き寄せます。
•貴重なプロパティ洞察のための原材料としての巨大な空間データライブラリ。私たちの空間データライブラリの規模は、加入者のための洞察を導き出す上で重要な利点です。私たちの空間データライブラリは、Cortexの重要な基盤となり、Matterportが低コストのデジタルおよびスマートフォンカメラを含むさまざまなカメラ技術を使用して強力な3Dデジタルツインを作成できるようにします。2022年12月31日現在、私たちのデータは約920万の空間から管理され、約280億平方フィートがキャプチャされています。その結果、私たちは不動産の洞察と分析を新たなレベルに引き上げ、さまざまな業界の加入者に利益をもたらしています。たとえば、施設管理者は建物のレイアウトを作成するのに必要な時間を大幅に短縮し、現地調査や実測モデルのコストを大幅に削減します。AECの加入者は、各実測空間の分析を使用して文書化を効率化し、簡単にコラボレーションできます。
•グローバルなリーチとスケール。私たちは、AI駆動の空間データプラットフォームを世界中に拡大し続けることに注力しています。私たちは、北米、ヨーロッパ、アジアに重要な存在感を持ち、各地域にリーダーシップチームと市場投入インフラを持っています。私たちはロンドン、シンガポール、アメリカ合衆国のいくつかの都市にオフィスを持ち、国際的な拡大を加速しています。2022年12月31日現在、私たちは170カ国以上で701,000人以上の加入者を持っています。私たちの空間データプラットフォームの地理的に無関係な性質は、国際的に成長し続ける上で重要な利点であると信じています。
•10年間の研究開発と革新を支える広範な特許ポートフォリオ。2022年12月31日現在、私たちは54件の特許が発行され、37件の特許出願が保留中です。私たちの成功は、ほぼ10年間の革新への注力に基づいています。革新はMatterportの中心にあり、私たちは市場のリーディングポジションをさらに強化するために研究開発への投資を優先し続けます。
•優れたキャプチャ技術。Matterportのキャプチャ技術プラットフォームは、建物や空間のMatterportデジタルツインを作成するために必要なさまざまなキャプチャデバイスをサポートするソフトウェアフレームワークです。
これには、LiDARカメラ、360カメラ、スマートフォン、Matterport Axis、Matterport Pro2およびPro3カメラのサポートが含まれます。Pro2カメラは、私たちの空間データの利点の基盤となり、Matterport対応のサードパーティキャプチャデバイスの配列でその利点を拡大しました。2022年8月、私たちはPro3カメラの発売を開始し、業界をリードするデジタルツインクラウドプラットフォームに大規模なアップデートを行いました。Matterport Pro3カメラは、高速な起動時間、交換可能なバッテリー、軽量設計を備えた高度な3Dキャプチャデバイスです。Pro3カメラは、屋内外の両方で機能し、速度、忠実度、多様性、精度のために設計されています。Pro2カメラとともに、私たちはPro3カメラの将来の販売が私たちのソリューションの採用を引き続き促進すると期待しています。Matterportは3Dキャプチャ体験を民主化し、高忠実度で高精度な3Dデジタルツインを、あらゆる建物タイプや加入者のニーズに対して容易に利用できるようにしています。他の3Dキャプチャソリューションプロバイダーは存在しますが、真の寸法的に正確な3D結果を生成できるものは非常に少なく、フォトリアリスティックな3Dで最終製品を自動的に作成できるものはさらに少ないです。この広範なキャプチャ技術の提供は、スマートフォンや360カメラでキャプチャされた写真から自動的に正確なデジタルツインを生成するために、AI駆動のCortexエンジンをトレーニングするために利用可能な豊富な空間データライブラリがなければ実現できません。
</document>"""
